# Converting Homestuck Collection data to an agnostic dataset

This notebook has all the steps for taking data from the [Unofficial Homestuck Collection](https://bambosh.dev/unofficial-homestuck-collection/)'s asset pack and converting it into a dataset that can be used for training a machine learning model.

Some planned uses for this are:
- Summarization
- Style transfer/LORAs
- Chatbots

# Constants and Imports

In [399]:
"""
IMPORTS
Put all at the beginning because I hate notebooks so much
"""
import os
import pandas as pd
import numpy as np
import json
import re
import random
import base64

from openai import OpenAI

from dotenv import load_dotenv

In [400]:

"""
CONSTANTS
This will extract constants from the env variables set in the .env file 
and make them accessible to the notebook
"""
print("Loading variables from .env file\n...")
load_dotenv()

ASSET_PACK_FOLDER = os.getenv("ASSET_PACK_FOLDER")
OUTPUT_FOLDER = os.getenv("OUTPUT_FOLDER")
OPENAI_API_KEY = os.getenv("OPENAI_API")
MODEL_ID = os.getenv("MODEL_ID")
print("Loaded variables successfully")
    
print("Loading constants\n...")
# Relevant folders and files

# Bespoke input files that aren't available elsewhere
INPUT_FOLDER = os.path.join(os.path.dirname(os.path.abspath('')), "input")

# Transcripts and commentary from ReadMSPA, assembled by Bambosh, Makin and Giovanh
MSPA_COMMENTARY = os.path.join(INPUT_FOLDER, "commentary.json")

# Full transcripts from ReadMSPA, assembled by Giovanh and Bambosh
READMSPA_ALTNARRATIVE = os.path.join(INPUT_FOLDER, "altnarrative.json")
READMSPA_ALTTEXT = os.path.join(INPUT_FOLDER, "alttext.json")
READMSPA_TRANSCRIPTS = os.path.join(INPUT_FOLDER, "transcripts.json")

# Panel tags from the Homestuck Search Engine
HSSE_TAGS = os.path.join(INPUT_FOLDER, "hsse_tags.json")
HSSE_SEARCH = os.path.join(INPUT_FOLDER, "hsse_search.json")

# POV cam data folder with txt files
POV_CAM_FOLDER = os.path.join(INPUT_FOLDER, "readable_timelines")

# Homestuck Collection's asset pack data folder
COLLECTION_DATA_FOLDER = os.path.join(ASSET_PACK_FOLDER, "archive/data")

# Holds all the text in MS Paint Adventures, including Homestuck
MSPA_TEXT_JSON = os.path.join(COLLECTION_DATA_FOLDER, "mspa.json")
# Holds the text for news posts
NEWS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "news.json")
# Holds the text for social media posts
SOCIAL_JSON = os.path.join(COLLECTION_DATA_FOLDER, "social.json")
# Most of this is irrelevant, but holds images for additional Hussie comics
# such as Team Special Olympics
ADDITIONAL_COMICS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "comics.json")
# Holds panels
PANELS_FOLDER = os.path.join(ASSET_PACK_FOLDER, "storyfiles")
HS_PANELS_FOLDER = os.path.join(PANELS_FOLDER, "hs2")

print("Loaded constants successfully")
      
openai_client = OpenAI(api_key=OPENAI_API_KEY)

print(f"OpenAI client loaded with model {MODEL_ID}")

Loading variables from .env file
...
Loaded variables successfully
Loading constants
...
Loaded constants successfully
OpenAI client loaded with model gpt-4o


# Extract MSPA data from Asset Pack

First of all, we want to open the MSPA_TEXT_JSON and fetch all the text data from it. This will be the main source of text data for our dataset.

In [401]:
"""
MSPA_TEXT_JSON format:
{
  "story": {  # The text in MS Paint Adventures, the comics themselves
    "000006": {
      "title": "Look for keyhole",
      "pageId": "000006",
      "timestamp": "1180921880",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure04.gif",
        "/advimgs/jb/mspaintadventure04b.gif"
      ],
      "content": "",
      "next": [
        "000008"
      ],
      "previous": "000005",
      "theme": "retro"
    },
    "000009": {
      "title": "Loudly tell that guy to pick up key and try it on the door.",
      "pageId": "000009",
      "timestamp": "1180931172",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure06.gif"
      ],
      "content": "Despite your bellowing, the man casually opens the door and leaves.",
      "next": [
        "000010"
      ],
      "previous": "000008",
      "theme": "retro"
    },
  },...
  "ryanquest": {...},  # Additional Ryanquest comic
  
  "faqs": {
    "general": {
      "title": "General FAQ - MS Paint Adventures",
      "pageId": "general",
      "content": "..." # html
    },
    "new": {...},  # New reader guide
    "science": {...},  # Science FAQ
    "sales": {...}, # This one was probably not Hussie, so ignore
  },
  "psExtras": {...}, # Bonus pages for Problem Sleuth with multiple images, same format with "title", "pageId" and "content" as the subkeys in "faqs"
  "wv": {...}, # "Exile" Homestuck pages with multiple images, same format as psExtras
  # Other keys are fully irrelevant
"""

def get_regular_story_records(mspa_data: dict, section_name: str):
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': page_content.get('timestamp', ''),
            'media': page_content.get('media', []),
            'next': page_content.get('next', []),
            'previous': page_content.get('previous', ''),
            'theme': page_content.get('theme', ''),
            'flag': page_content.get('flag', []),
        }
        records.append(record)
    return records

def get_media_from_html_content(content: str):
    # Extract media from HTML content
    media = []
    # Extract media URLs from the content
    matches = re.findall(r'src="([^"]+)"', content)
    for match in matches:
        # Remove the string "http://www.mspaintadventures.com" and "https://www.mspaintadventures.com" from the beginning if it exists
        match_cleaned = re.sub(r'^https?://www\.mspaintadventures\.com', '', match)
        media.append(match_cleaned)
    return media

def get_onepage_story_records(mspa_data: dict, section_name: str):
    # We need this one for faqs/psExtras/wv, which keep all information in one page and need the panels manually parsed
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': None,
            'media': get_media_from_html_content(page_content.get('content', '')),
            'next': None,
            'previous': None,
            'theme': None,
            'flag': None,
        }
        records.append(record)
    return records

# Function to load and process MSPA text data from mspa.json
def load_mspa_text_data(mspa_json_path):
    """
    Loads and processes MSPA text data from the given JSON file.

    Args:
        mspa_json_path (str): Path to the mspa.json file.

    Returns:
        pd.DataFrame: A DataFrame containing the MSPA text data.
    """
    import json
    import pandas as pd

    # Load the JSON data
    with open(mspa_json_path, 'r', encoding='utf-8') as f:
        mspa_data = json.load(f)

    # Initialize an empty list to store records
    records = []
    
    # Print unique sections
    print(f"Unique sections in mspa.json: {list(mspa_data.keys())}")

    # Sections to process (basically we just skip FAQs)
    records += get_regular_story_records(mspa_data, 'story')
    records += get_regular_story_records(mspa_data, 'ryanquest')
    records += get_onepage_story_records(mspa_data, 'wv')
    records += get_onepage_story_records(mspa_data, 'psExtras')

    # Create a DataFrame from the records
    df = pd.DataFrame(records)

    return df

# Now call the function and store the result in a DataFrame
mspa_df = load_mspa_text_data(MSPA_TEXT_JSON)
mspa_df

Unique sections in mspa.json: ['story', 'ryanquest', 'psExtras', 'wv', 'faqs', 'damara', 'oilRetcon']


,section,pageId,title,content,timestamp,media,next,previous,theme,flag
0,story,000006,Look for keyhole,,1180921880,"[/advimgs/jb/mspaintadventure04.gif, /advimgs/...",[000008],000005,retro,[]
1,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,[/advimgs/jb/mspaintadventure06.gif],[000010],000008,retro,[]
2,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,"[/advimgs/jb/mspaintadventure07.gif, /advimgs/...",[000011],000009,retro,[]
3,story,000011,Open door,Wonders where the fuck that pumpkin went???,1180931414,[/advimgs/jb/mspaintadventure08.gif],[000012],000010,retro,[]
4,story,000005,Grow two muscular arms with which the manhandl...,"He already has arms, stupid!",1180921870,[/advimgs/jb/mspaintadventure03.gif],[000006],000004,retro,[]
...,...,...,...,...,...,...,...,...,...,...
10049,psExtras,ps000037,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000037_1.gif, /extras/ps000037_2.gi...",None,None,None,None
10050,psExtras,ps000039,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000039_1.gif, /extras/ps000039_2.gif]",None,None,None,None
10051,psExtras,ps000034,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000034_1.gif, /extras/ps000034_2.gi...",None,None,None,None
10052,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000031_1.gif, /extras/ps000031_2.gi...",None,None,None,None


In [402]:
# Show pageId == anunsealedrecord, a special case
mspa_df[mspa_df['pageId'] == 'anunsealedtunnel']

,section,pageId,title,content,timestamp,media,next,previous,theme,flag
10006,wv,anunsealedtunnel,MS Paint Adventures,"\n\n <tbody><tr>\n <td bgcolor=""#FFFFFF""...",None,[/storyfiles/hs2/waywardvagabond/anunsealedtun...,None,None,None,None


# Creating vizPageId from pageId

The Homestuck Collection uses the MSPA page IDs, but the Homestuck website uses its own page IDs. We can convert the MSPA page IDs to the Homestuck website page IDs with some clever and totally not bruteforced heuristics. We will need to do this sooner or later because multiple upcoming datasets use the Viz page ids, which are missing in mspa_df right now.

In [403]:
# Define the adventure number to story name mapping
adventure_mapping = {
    0: "Other Section",
    1: "Jailbreak",
    2: "Bard Quest",
    3: "Blood Spade",
    4: "Problem Sleuth",
    5: "Homestuck BETA",
    6: "Homestuck",
    "ryanquest": "Ryanquest",
    "psExtras": "Problem Sleuth Extras"
}

# Define the story name to URL path mapping
story_url_mapping = {
    "Jailbreak": "jailbreak",
    "Bard Quest": "bard-quest",
    "Blood Spade": "blood-spade",
    "Problem Sleuth": "problem-sleuth",
    "Homestuck BETA": "beta",
    "Homestuck": "story",
    "Ryanquest": "ryanquest",
    "Other Section": "story/waywardvagabond"
}

def get_story_num(page_id):
    """
    Determines the story ID based on the page number.
    
    Args:
        page_id (str): The pageId, can be numeric or special identifier.
    
    Returns:
        int or str: The story number (1-6) or 'Unknown'
    """
    if page_id.isdigit():
        page_num = int(page_id.lstrip('0'))
        if page_num <= 135:
            return 1  # Jailbreak
        elif 136 <= page_num <= 216:
            return 2  # Bard Quest
        elif 219 <= page_num <= 1892:
            return 4  # Problem Sleuth
        elif 1893 <= page_num <= 1900:
            return 5  # Homestuck BETA
        elif 1901 <= page_num <= 10030:
            return 6  # Homestuck
        else:
            return "Unknown"
    else:
        # Handle special cases
        if page_id == 'mc0001':
            return 3  # Blood Spade
        elif page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
            return 6  # Homestuck
        else:
            return "Unknown"

def mspa_to_viz(row):
    """
    Converts a row's pageId and section to vizStory, vizPageNumber, vizPageId.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        pd.Series: A series with 'vizStory', 'vizPageNumber', and 'vizPageId'.
    """
    section = row['section'].lower()
    page_id = row['pageId']
    
    if section == 'ryanquest':
        viz_story = "Ryanquest"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"ryanquest/{viz_page_number}" if isinstance(viz_page_number, int) else f"ryanquest/{viz_page_number}"
    elif section == 'story':
        story_num = get_story_num(page_id)
        if isinstance(story_num, int) and story_num in adventure_mapping:
            viz_story = adventure_mapping[story_num]
            # Adjust vizPageNumber based on story_num
            try:
                if story_num == 1:  # Jailbreak
                    viz_page_number = int(page_id.lstrip('0'))
                elif story_num == 2:  # Bard Quest
                    viz_page_number = int(page_id.lstrip('0')) - 135
                elif story_num == 3:  # Blood Spade
                    viz_page_number = "1"  # Only 'mc0001'
                elif story_num == 4:  # Problem Sleuth
                    viz_page_number = int(page_id.lstrip('0')) - 218
                elif story_num == 5:  # Homestuck BETA
                    viz_page_number = int(page_id.lstrip('0')) - 1892
                elif story_num == 6:  # Homestuck
                    if page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
                        viz_page_number = page_id  # Preserve special strings
                    else:
                        viz_page_number = int(page_id.lstrip('0')) - 1900
                else:
                    viz_page_number = None
            except:
                viz_page_number = None
            
            # Construct vizPageId based on vizPageNumber
            if isinstance(viz_page_number, int):
                viz_page_id = f"{story_num}/{viz_page_number}"
            elif isinstance(viz_page_number, str):
                viz_page_id = f"{story_num}/{viz_page_number}"
            else:
                viz_page_id = f"{story_num}/000000"
        else:
            viz_story = "Unknown Adventure"
            viz_page_number = None
            viz_page_id = "Unknown/000000"
    else:
        viz_story = "Other Section"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"other/{viz_page_number}" if isinstance(viz_page_number, int) else f"other/{viz_page_number}"
    
    return pd.Series({
        'vizStory': viz_story,
        'vizPageNumber': viz_page_number,
        'vizPageId': viz_page_id
    })

def create_viz_link(row):
    """
    Creates a visualization link based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed URL or None if information is missing.
    """
    viz_story = row['vizStory']
    viz_page_number = row['vizPageNumber']
    
    if pd.isnull(viz_story) or pd.isnull(viz_page_number):
        return None
    
    base_url = "https://homestuck.com/"
    story_path = story_url_mapping.get(viz_story, "other")
    
    # Handle potential edge cases where viz_page_number is not an integer
    if isinstance(viz_page_number, float):
        if viz_page_number.is_integer():
            viz_page_number = int(viz_page_number)
        else:
            # If it's not an integer, handle accordingly (e.g., set to None or round)
            return None
    
    return f"{base_url}{story_path}/{viz_page_number}"

mspa_df['pageId'] = mspa_df['pageId'].astype(str)

# Apply the mspa_to_viz function to create 'vizStory', 'vizPageNumber', and 'vizPageId' columns
mspa_df[['vizStory', 'vizPageNumber', 'vizPageId']] = mspa_df.apply(mspa_to_viz, axis=1)

# Create the 'vizLink' column
mspa_df['vizLink'] = mspa_df.apply(create_viz_link, axis=1)

# Convert 'vizPageNumber' to integer where possible, else keep as string for special cases
def convert_viz_page_number(x):
    if isinstance(x, int):
        return x
    elif isinstance(x, str):
        return x  # Preserve special strings
    else:
        return pd.NA

mspa_df['vizPageNumber'] = mspa_df['vizPageNumber'].apply(convert_viz_page_number)

# Display the updated DataFrame sample
mspa_df[['section', 'title', 'pageId', 'content', 'vizStory', 'vizPageNumber', 'vizPageId', 'vizLink']].sample(10)

,section,title,pageId,content,vizStory,vizPageNumber,vizPageId,vizLink
9704,story,[A6A6I5] ====>,009741,"|DIALOGLOG|<br /><span style=""color: #005682"">...",Homestuck,7841,6/7841,https://homestuck.com/story/7841
4717,story,Jade: Head home.,004757,Someone has connected as your server player.<b...,Homestuck,2857,6/2857,https://homestuck.com/story/2857
8963,story,[S][A6A6I4] ====>,009001,,Homestuck,7101,6/7101,https://homestuck.com/story/7101
1677,story,Next.,001717,"You consume your counterparts, ZAD and FAD, fi...",Problem Sleuth,1499,4/1499,https://homestuck.com/problem-sleuth/1499
7823,story,[A6I5] ==>,007864,"|DIALOGLOG|<br /><span style=""color: #005682"">...",Homestuck,5964,6/5964,https://homestuck.com/story/5964
4508,story,Karkat: Troll this worthless human.,004528,|PESTERLOG|<br />carcinoGeneticist <span style...,Homestuck,2628,6/2628,https://homestuck.com/story/2628
7311,story,[A6I4] ==>,007352,Yeah.,Homestuck,5452,6/5452,https://homestuck.com/story/5452
2623,story,Retrieve arms from...,002663,THEY'RE RIGHT THERE.<br /><br />IN PLAIN SIGHT...,Homestuck,763,6/763,https://homestuck.com/story/763
5940,story,[o] ==>,005981,"<span style=""color: black"">--------------- <im...",Homestuck,4081,6/4081,https://homestuck.com/story/4081
5401,story,==>,005442,,Homestuck,3542,6/3542,https://homestuck.com/story/3542


# Unpacking the page-based dataset into a panel-based dataset

The previous works fine for data sourced on pages instead of panels, like commentaries, but we're merging with ReadMSPA next, so we will need to `explode` the data and do some more edits, since the panel srcs will need some editing to match the transcripts.

In [404]:
mspa_panels_df = mspa_df.explode('media').reset_index(drop=True)
# Rename 'media' to 'src' for clarity
mspa_panels_df.rename(columns={'media': 'src'}, inplace=True)
# Get a sample of Homestuck panels
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6


In [405]:
# Some panel transcripts will have an id format like `00001.gif.transcript`, which in our current src is `/storyfiles/hs2/00001.gif`. We need to make a transcriptSrc to match the panel src. This applies no matter what the comics is, so we can't assume it will be /storyfiles/hs2/... we just pick the actual image file name and add '.transcript' to it.
def create_transcript_src(src):
    if pd.isnull(src):
        return None
    
    # Extract the image file name from the src
    image_file = src.split('/')[-1]
    # Append '.transcript' to the image file name
    transcript_src = f"{image_file}.transcript"
    
    return transcript_src
    
# Apply the function to create 'transcriptSrc' column
mspa_panels_df['transcriptSrc'] = mspa_panels_df['src'].apply(create_transcript_src)
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript


In [406]:
# Now we need to do something similar for MSPA URLs to match the key format for pages of the READMSPA format. We have a pageId like 001901, but we need to get the story_num using the matching we already have, and prepend that to the page number. So 001901 would become '6/001901' because that page number matches Homestuck. This can be the new column "pageNumber" because it's the vizPageNumber in MSPA format..
def create_page_number(row):
    """
    Creates a page number based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed page number or None if information is missing.
    """
    
    viz_page_number = row['pageId']
    
    if pd.isnull(viz_page_number):
        return None
        
    viz_story_number = get_story_num(str(viz_page_number))
    
    return f"{viz_story_number}/{viz_page_number}"

# Apply the function to create 'pageNumber' column
mspa_panels_df['pageNumber'] = mspa_panels_df.apply(create_page_number, axis=1)

# Display the updated DataFrame sample
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc,pageNumber
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript,6/001902
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript,6/001901
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript,6/001904
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript,6/001903
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript,6/001908
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript,6/001905
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript,6/001909
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript,6/001907
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript,6/001907
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript,6/001906


# Getting panel transcripts from ReadMSPA plugin

ReadMSPA's data (and its plugin from the collection) comes with text transcripts of every image's text, if not descriptions. We can use that.

In [407]:
"""
The ReadMSPA data is divided in three similar files for unknown reasons, READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT, READMSPA_ALTNARRATIVE. The latter tends to have way longer transcriptions, but there are long transcriptions in READMSPA_ALTTEXT, and they have different formats besides. READMSPA_TRANSCRIPTS even contains transcripts of youtube links linked in the pages, which we should preserve anyway. 
Note the keys are not equivalent to pageId. These keys use the mspaintadventures.com format, where page numbers were padded with zeroes, as well as sharing a count for every adventure in the site, from Jailbreak to Homestuck. Homestuck's first page is 001901 and it's the adventure number 6 (instead of 4 for historical reasons, but this is not really relevant to us right now).

Portion of READMSPA_TRANSCRIPTS (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
{
    "6/001901": [
        {
            "src": null,
            "alt": "SBURB BETA",
            "contents": "SBURB BETA",
            "id": "00001.gif.transcript",
            "data": null,
            "class": [
                "transcript"
            ],
            "style": "background: white; color: #C1C1C1",
            "tag": "div"
        }
    ],
    "6/010030": [
        {
            "src": "http/www.youtube.com-rmzu89jy2j8.mp4",
            "alt": "* (This transcribes both the snapchat pics from the right of the video - which as narrative, are included in readmspa.org word count stats - and the credits from the left, which are not.)\n\n\nmspaofficial\n\nAdded Me\nAdd Friends\nMy Friends\n\n\n* HOMESTUCK\n\n* BY\n* ANDREW HUSSIE\n\n\nGreetings From\nEARTH C\n\n\n* SOUND\n* CONTRIBUTORS (A-Z)\n\n* Alex Rosetti\n* Andrew Huo\n* BurnedKirby\n* Charlie Clouser\n* Clark Powell\n* Curt Blakeslee\n* David DeCou\n* David Ko\n\n\nMAYOR\n\n\n* Dianne Warren\n* DJ Sai Tae\n* Eddie Morton\n* Eligecos\n* Erik Scheele\n* Eston Schweickart\n* First Turn Fold\n* Frank Haught\n* Gabe Nezovic\n\n\n* Insane Clown Posse\n* James Roach\n* Jan Van Den Hemel\n* Joseph Aylsworth\n* Kalibration\n* Kera Jones\n* Kevin Regamey\n* Kezinox\n* Malcolm Brown\n\n\n* Mark Hadley\n* Michael Guy Bowman\n* Michael Vallejo\n* Nick Smalley\n* Noel Sadwin\n* Paul Tuttle Starr\n* Perry Sullivan\n* Plumegeist\n* Robert J! Lake\n\n\n* Samm 413\ntentacleTherapist\ngallowsCallibrator\ntipsyGnostalgic\ngrimAuxiliatrix\ngolgothasTerror\ntimaeusTestified\ngutsyGumshoe\n\n\nCALIBORN: COME AT ME BRO.\n\n\nJOHN: step off.\n\n\nCALIBORN: COME TO MY DARK CARNIVAL. \"MOTHER FUCKER\".\n\n\nJOHN: i'll do it...\n\n\nCALIBORN: MAKE A MOVE. AND THE BUNNY GETS IT.",
            "contents": "",
            "id": "08123.mp4",
            "data": "End Credits",
            "class": [
                "flash",
                "alt-narrative"
            ],
            "style": null,
            "tag": "embed"
        }
    ],
```

Portion of READMSPA_ALTTEXT (a key matches the src column):
```json
{
    "advimgs/jb/mspaintadventure06.gif": "\nBELLOW!\n\nSLAM\n",
    "advimgs/jb/mspaintadventure09d.gif": "CRACK",
    "advimgs/jb/mspaintadventure09e.gif": "SLAM",
    "advimgs/jb/mspaintadventure05.gif": "CLANK",
    "advimgs/jb/mspaintadventure08.gif": "? ? ? ?",
    ...
    "storyfiles/hs2/07655.gif": "\nDAVE: ...\nDIRK: ...\n",
    "storyfiles/hs2/07651.gif": "nuzzzzzzzzzle",
    "storyfiles/hs2/07650.gif": "nuzzzzle",
    "storyfiles/hs2/07648.gif": "KANAYA: Shoutpole",
}

Portion of READMSPA_ALTNARRATIVE (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
"6/002153": [
        {
            "src": null,
            "alt": "<style>.page[id=\"6/002153\"] .walkaround p { border-color: #23CE27; border-radius: 0;\n\t\t       \t    \t\t    background-color: white; max-width: 100%; }</style>\n* Derived in part from <a href=\"http://pastebin.com/a3k6RgMR\">a transcript by ShadowOfFate</a>\n\n<div style=\"padding: 1em; background-color: white; border: solid 3px black\">CLICK THIS</div><dl>\n<dt>* click it!</dt>\n<dd>\n<p>To walk around, use the mouse, arrow keys, or WASD keys. Click on various objects to open command menus for them!\n\nOutstanding Flash programming by Alexis 'Gankro' Beingessner.</p>\n</dd>\n\n<dt>* click jestersprite</dt>\n<dd>\n<kbd>&gt; WHAT'S THAT\n</kbd>\n<p>It looks different now.\n\nAfter you bit that APPLE, your whole house seemed to be trasported somewhere. Then the APPLE disappeared and the KERNELSPRITE underwent a transformation. Aside from the change in appearance, the transformation doesn't seem to have any relevant ramifications. You still can't understand a word this idiot says.</p>\n<kbd>&gt; THE GHOST CLOWN...",
            "id": "00253.swf.transcript",
            "data": "John explores his house with WV (transcript and walkthrough)"
        }
    ],
    "6/002149": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://mspaintadventures.wikia.com/wiki/Rose%27s_Walkthrough\">MSPA Wiki</a>\n\n<p style=\"background-color: black; color: white; padding: 1em;\n          font-family: Lucida Console, courier;\"><b>Sburb Beta FAQ/Walkthrough</b> by <span style=\"color: #ff6600\">tentacleTherapist</span>\n<b>Hosted by</b> <span style=\"color: #ff6600\">GameFAQs</span>\nVersion 1.0, Last Updated 2009-04-13\n<span style=\"color: #ff6600\">View/Download Original File</span>\nLiked this FAQ? Click to <span style=\"color: #ff6600\">recommend</span> it to other GameFAQs users.\n</p>\n<article style=\"font-family: Lucids Console, courier\">\nSburb Beta Walkthrough\nVersion 1.0, April 13, 2009\nBy tentacleTherapist\n\n=============================== TABLE OF CONTENTS ===============================\n\n1. Caveats and Condolences........................... [0000]\n2. Walkthrough (Incomplete).......................... [A000]\n2.1. An Examination of the Basics.................... [A000]\n2.2. So your cruxtruder is ticking. Do this to live.. [A100]\n2.3. The Long and Short. The Medium too.............. [B100]\nsome stuff about captcha codes and punch card alchemy [Z001]\nC. Appendix 3 -- Screen Captures, pt. 1.............. [Z301]\n?. Rose: Egress...",
            "id": "00249_2.gif.transcript",
            "data": "Rose's Sburb Beta Walkthrough - Caveats and Condolences"
        }
    ],
    "6/002037": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://pastebin.com/FQhpyeKx\">a transcript by ShadowOfFate</a>\n\n<ul style=\"list-style: none; background-color: #4CE24E; padding: 1em;\n    color: white; text-align: center; white-space: normal\">\n<li>Transforming Soffits</li>\n<li>Reorganizing Keys</li>\n<li>Formalizing Immersion Joints</li>\n<li>Justifying Kick Extractors</li>\n<li>Advising Aggregates</li>\n<li>Managing Elbows</li>...",
            "id": "00137.swf.transcript",
            "data": "Sburb Beta loading screen"
        }
    ]
}

Our objective here is to combine the information so that, for each panel, we'll have its text transcript.
"""

# Extract the ReadMSPA data from the JSON files
def load_readmspa_data(transcripts_path, alttext_path, altnarrative_path):
    """
    Loads and processes the ReadMSPA data from the given JSON files.

    Args:
        transcripts_path (str): Path to the transcripts JSON file.
        alttext_path (str): Path to the alttext JSON file.
        altnarrative_path (str): Path to the altnarrative JSON file.

    Returns:
        pd.DataFrame: A DataFrame containing the ReadMSPA data.
    """
    # Load the JSON data
    with open(transcripts_path, 'r', encoding='utf-8') as f:
        transcripts_data = json.load(f)
    with open(alttext_path, 'r', encoding='utf-8') as f:
        alttext_data = json.load(f)
    with open(altnarrative_path, 'r', encoding='utf-8') as f:
        altnarrative_data = json.load(f)

    # Initialize an empty list to store page records
    page_records = []
    
    # Iterate over each data dictionary of READMSPA_TRANSCRIPTS and READMSPA_ALTNARRATIVE
    for data_dict in [transcripts_data, altnarrative_data]:
        # Iterate over each key-value pair
        for key, value_list in data_dict.items():
            # Extract relevant fields
            for value in value_list:
                record = {
                    'pageNumber': key,
                    'src': value.get('src', None),
                    'contents': value.get('contents', ''),
                    'transcriptSrc': value.get('id', ''),
                    'data': value.get('data', ''),
                    'class': value.get('class', []),
                    'style': value.get('style', ''),
                    'tag': value.get('tag', ''),
                    'alt': value.get('alt', ''),
                }
                page_records.append(record)
            
    # Initialize an empty list to store alttext records
    panel_records = []
    
    # Iterate over each key-value pair in READMSPA_ALTTEXT
    for key, value in alttext_data.items():
        # Extract relevant fields
        record = {
            'src': key,
            'contents': value,
        }
        panel_records.append(record)
        
    # Create DataFrames from the records
    page_df = pd.DataFrame(page_records)
    panel_df = pd.DataFrame(panel_records)
    
    return page_df, panel_df

# Now call the function and store the results in DataFrames
readmspa_pages_df, readmspa_panels_df = load_readmspa_data(READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT, READMSPA_ALTNARRATIVE)

# Display the updated DataFrame samples
readmspa_pages_df.sample(5)
    

,pageNumber,src,contents,transcriptSrc,data,class,style,tag,alt
198,6/005552,storyfiles/hs2/03649_1.gif,,03649_1.gif,AGAIN with the metric sytstem,[],,,AGAIN with the\nmetric sytstem what IS it even...
86,6/006869,None,"<p style=""color: red"">Booyeah.\n\nDouble\n\nro...",GLNHH.transcript,None,"[transcript, postlink]",,div,"<p style=""color: red"">Booyeah.\n\nDouble\n\nro..."
158,6/002340,None,"* Derived from <a href=""http://mspaintadventur...",00440_2.gif.transcript,Rose's Sburb Beta Walkthrough - The Long and S...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
138,6/003258,None,"* Derived in part from <a href=""http://pastebi...",01358.swf.transcript,John explores LOWAS with help from Nannasprite...,"[transcript, alt-narrative, walkaround]",,div,"* Derived in part from <a href=""http://pastebi..."
10,6/009736,None,"<span style=""color: #0715cd; font-size: 400%;""...",07832_4.gif.transcript,None,[transcript],,div,"<span style=""color: #0715cd; font-size: 400%;""..."


In [408]:
readmspa_panels_df.sample(5)

,src,contents
1537,storyfiles/hs2/03116.gif,VRISKA: KILL ME
762,storyfiles/hs2/05201.gif,DIRK: _
2615,storyfiles/hs2/00368.gif,\nwww\nwww\nwww\nwww\n
1384,storyfiles/hs2/03475.gif,\npap\n\npoooke\n
3386,advimgs/jb/lv3/mspaintadventure20e.gif,GUSH


In [409]:
# Now we can merge readmspa_pages_df on its id (which will match transcriptSrc) with mspa_panels_df on transcriptSrc
# Note we only care about 
mspa_panels_df = mspa_panels_df.merge(readmspa_pages_df, on='transcriptSrc', how='left', suffixes=('', '_readmspapages'))
# Drop duplicate rows, we just want one record per "src".
mspa_panels_df.drop_duplicates(subset='src', inplace=True)
# Show some panels with transcripts
mspa_panels_df[~mspa_panels_df['contents'].isnull()].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcriptSrc,pageNumber,pageNumber_readmspapages,src_readmspapages,contents,data,class,style,tag,alt
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],...,00002.gif.transcript,6/001902,6/001902,None,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\..."
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],...,00001.gif.transcript,6/001901,6/001901,None,SBURB BETA,None,[transcript],background: white; color: #C1C1C1,div,SBURB BETA
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],...,00003.gif.transcript,6/001903,6/001903,None,"<b style=""color: green"">✓</b> <b style=""color:...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: green"">✓</b> <b style=""color:..."
2408,story,001912,John: Read note on drawer.,This note is rich with the aromas of FATHERLY ...,1239679025,/storyfiles/hs2/00012.gif,[001913],001911,,[],...,00012.gif.transcript,6/001912,6/001912,None,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.,None,"[transcript, dad-note]",,div,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.
2562,story,002037,[S] ==>,,1241865178,/storyfiles/hs2/00137/00137.swf,[002038],002036,,"[F, HQ]",...,00137.swf.transcript,6/002037,6/002037,None,"* Derived from <a href=""http://pastebin.com/FQ...",Sburb Beta loading screen,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://pastebin.com/FQ..."
2694,story,002149,ACT 2 ==>,,1244666782,/storyfiles/hs2/00249_2.gif,[002150],002148,,[],...,00249_2.gif.transcript,6/002149,6/002149,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - Caveats and Co...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2704,story,002153,[S] YOU THERE. BOY.,,1244949337,/storyfiles/hs2/00253/00253.swf,[002154],002152,,[F],...,00253.swf.transcript,6/002153,6/002153,None,"<style>.page[id=""6/002153""] .walkaround p { bo...",John explores his house with WV (transcript an...,"[transcript, alt-narrative, walkaround]",,div,"<style>.page[id=""6/002153""] .walkaround p { bo..."
2726,story,002172,==>,,1245374550,/storyfiles/hs2/00272_2.gif,[002173],002171,,[],...,00272_2.gif.transcript,6/002172,6/002172,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - An Examination...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2919,story,002340,==>,"She's not finished with this yet! Jeez, cut he...",1249444721,/storyfiles/hs2/00440_2.gif,[002341],002339,,[],...,00440_2.gif.transcript,6/002340,6/002340,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - The Long and S...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2998,story,002410,==>,"<a href=""http://www.mspaintadventures.com/stor...",1250822032,/storyfiles/hs2/00510_2.gif,[002411],002409,,[],...,00510_2.gif.transcript,6/002410,6/002410,None,"<article style=""font-family: Lucida Console, c...",Rose's Sburb Beta Walkthrough - Screen Captures,"[transcript, alt-narrative]",,div,"<article style=""font-family: Lucida Console, c..."


In [410]:
# Merge the panel data too, this time on readmspa_panels_df's src, which matches mspa_panels_df's src, but, critically, without the slash at the beginning. 
# Make a copy of the src column in readmspa_panels_df to match the src column in mspa_panels_df
mspa_panels_df['src_copy'] = mspa_panels_df['src'].apply(lambda x: x[1:] if x.startswith('/') else x)
# Merge the DataFrames
mspa_panels_annotated_df = mspa_panels_df.merge(readmspa_panels_df, left_on='src_copy', right_on='src', how='left', suffixes=('', '_readmspanels'))
# Show a panel with annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,src_readmspapages,contents,data,class,style,tag,alt,src_copy,src_readmspanels,contents_readmspanels
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,storyfiles/hs2/01085_2.gif,storyfiles/hs2/01085_2.gif,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [411]:
# Time to clean up, now that we know transcripts in either "alt" or "contents_readmspanels"
# Useless columns are "transcriptSrc", "pageNumber_readmspapages", "src_readmspapages", "contents", "data", "src_copy", "src_readmspanels"
# "class", "style" and "tag" should be renamed to "transcript_class", "transcript_style" and "transcript_tag" respectively
# "transcript" should be a new column that contains the value of either "alt" or "contents_readmspanels"
# "alt" and "contents_readmspanels" should be dropped afterwards

# Drop the unnecessary columns
mspa_panels_annotated_df.drop(columns=['transcriptSrc', 'pageNumber_readmspapages', 'src_readmspapages', 'contents', 'data', 'src_copy', 'src_readmspanels'], inplace=True)

# Rename the 'class', 'style', and 'tag' columns
mspa_panels_annotated_df.rename(columns={'class': 'transcript_class', 'style': 'transcript_style', 'tag': 'transcript_tag'}, inplace=True)

# Create the 'transcript' column
mspa_panels_annotated_df['transcript'] = mspa_panels_annotated_df['alt'].fillna(mspa_panels_annotated_df['contents_readmspanels'])

# Drop the 'alt' and 'contents_readmspanels' columns
mspa_panels_annotated_df.drop(columns=['alt', 'contents_readmspanels'], inplace=True)

# Show a panel with cleaned annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],Homestuck,1085,6/1085,https://homestuck.com/story/1085,6/002985,NaN,NaN,NaN,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [412]:
mspa_panels_annotated_df

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
0,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
1,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04b.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
2,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,/advimgs/jb/mspaintadventure06.gif,[000010],000008,retro,[],Jailbreak,9,1/9,https://homestuck.com/jailbreak/9,1/000009,NaN,NaN,NaN,\nBELLOW!\n\nSLAM\n
3,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
4,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07b.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000031_3.gif,None,None,None,None,Other Section,ps000031,other/ps000031,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000031,NaN,NaN,NaN,NaN
11692,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_1.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11693,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_2.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11694,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_3.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN


# Getting panel tags from HSSE

The Homestuck Search Engine people tagged the tags of the first four acts, from characters to locations and more. Will be extremely useful for image transcription.

In [413]:
"""
HSSE_TAGS and HSSE_SEARCH file contain the entirety of the Homestuck Search Engine tagged data (only the first four acts, until page 1988 inclusive and excluding some swfs) in its own bespoke JSON format.
HSSE_TAGS is the simpler json with tag definitions, and which tags contain other tags:
```json
 "definitions": {
    "0": {
      "_id": 0,
      "name": "Character",
      "children": [
        1,
        32,
        56,
        60,
        104,
        132,
        148,
        155,
        173,
        184,
        253
      ]
    },
    "1": {
      "_id": 1,
      "name": "Human",
      "children": [
        2,
        15
      ]
    },
    "2": {
      "_id": 2,
      "name": "Kid",
      "children": [
        3,
        10
      ]
    },
    "3": {
      "_id": 3,
      "name": "Beta Kid",
      "children": [
        4,
        5,
        7,
        9
      ]
    },
    "4": {
      "_id": 4,
      "name": "John Egbert",
      "children": []
    },
    ...
}
```

HSSE_SEARCH is the more complex json with the actual tags for each panel:
```json
[
  {
    "_id": 0,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "url": "https://homestuck.com/story/1",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      749,
      801,
      1301,
      602,
      1192,
      711,
      1349
    ],
    "page": 1
  },
  {
    "_id": 1,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "url": "https://homestuck.com/story/2",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      1349,
      602
    ],
    "page": 2
  },
  ...
]
```
Our objective here is to combine the information so that, for each page, we'll have its human readable tags. 
"""

def load_hsse_tags(tags_path):
    """
    Loads and processes the HSSE tag definitions.

    Args:
        tags_path (str): Path to the hsse_tags.json file.

    Returns:
        dict: A dictionary mapping tag IDs (int) to tag names (str).
    """
    with open(tags_path, 'r', encoding='utf-8') as f:
        tags_data = json.load(f)
    
    # Extract the 'definitions' key
    definitions = tags_data.get('definitions', {})
    
    # Build mapping from _id (int) to name (str)
    tag_id_to_name = {}
    for tag_id_str, tag_info in definitions.items():
        tag_id = tag_info.get('_id')
        name = tag_info.get('name')
        if tag_id is not None and name is not None:
            tag_id_to_name[tag_id] = name
    return tag_id_to_name

def load_hsse_search(search_path):
    """
    Loads and processes the HSSE page-tag associations.

    Args:
        search_path (str): Path to the hsse_search.json file.

    Returns:
        dict: A dictionary mapping page IDs (str) to lists of tag IDs (int).
    """
    with open(search_path, 'r', encoding='utf-8') as f:
        search_data = json.load(f)
    
    # search_data is a list of dicts
    page_to_tag_ids = {}
    for entry in search_data:
        page = entry.get('page')  # integer
        tags = entry.get('tags', [])  # list of integers
        if page is not None:
            page_str = str(page)  # Convert page number to string to match 'pageId'
            page_to_tag_ids[page_str] = tags
    
    return page_to_tag_ids

def map_tags_to_names(page_to_tag_ids, tag_id_to_name):
    """
    Maps tag IDs to tag names for each page.

    Args:
        page_to_tag_ids (dict): Dictionary mapping page IDs to lists of tag IDs.
        tag_id_to_name (dict): Dictionary mapping tag IDs to tag names.

    Returns:
        dict: Dictionary mapping page IDs to lists of tag names.
    """
    page_to_tag_names = {}
    for page_id, tag_ids in page_to_tag_ids.items():
        # Ensure tag_ids is a list
        if not isinstance(tag_ids, list):
            tag_ids = [tag_ids]
        # Map each tag ID to its name, handle unknown tags gracefully
        tag_names = [tag_id_to_name.get(tag_id, f"Unknown Tag {tag_id}") for tag_id in tag_ids]
        page_to_tag_names[page_id] = tag_names
    return page_to_tag_names

def merge_tags_into_mspa_df(mspa_df, page_to_tag_names):
    """
    Merges HSSE tag data into the main MSPA DataFrame.

    Args:
        mspa_df (pd.DataFrame): The main MSPA DataFrame with transcripts.
        page_to_tag_names (dict): Dictionary mapping page IDs to lists of tag names.

    Returns:
        pd.DataFrame: The updated MSPA DataFrame with tags incorporated.
    """
    # Edit page_to_tag_names so the keys have the "6/" prefix to match the Viz page numbers
    page_to_tag_names = {f"6/{k}": v for k, v in page_to_tag_names.items()}
    
    # Create a Series from the page_to_tag_names dictionary
    tags_series = pd.Series(page_to_tag_names, name='tags')
    
    # Merge the tags into mspa_df, matching its vizPageId with the keys of the tags_series
    mspa_df_with_tags = mspa_df.merge(tags_series, left_on='vizPageId', right_index=True, how='left')
    
    # Replace NaN with empty lists for pages without tags
    mspa_df_with_tags['tags'] = mspa_df_with_tags['tags'].apply(lambda x: x if isinstance(x, list) else [])
    
    return mspa_df_with_tags

# Load HSSE tag definitions
tag_id_to_name = load_hsse_tags(HSSE_TAGS)
print(f"Loaded {len(tag_id_to_name)} tags from hsse_tags.json.")

# Load HSSE page-tag associations
page_to_tag_ids = load_hsse_search(HSSE_SEARCH)
print(f"Loaded tag associations for {len(page_to_tag_ids)} pages from hsse_search.json.")

# Map tag IDs to tag names
page_to_tag_names = map_tags_to_names(page_to_tag_ids, tag_id_to_name)
print("Mapped tag IDs to tag names.")

# Merge tags into the main MSPA DataFrame
mspa_df_with_tags = merge_tags_into_mspa_df(mspa_panels_annotated_df, page_to_tag_names)

# # Display the first few rows with tags not an empty list
mspa_df_with_tags[mspa_df_with_tags['tags'].apply(len) > 0].sample(10)

Loaded 1427 tags from hsse_tags.json.
Loaded tag associations for 8002 pages from hsse_search.json.
Mapped tag IDs to tag names.


,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags
4102,story,003341,==>,Once you're done you'll captchalogue the bottl...,1266688252,/storyfiles/hs2/01441_1_retcon.gif,[003342],003340,,[R4],Homestuck,1441,6/1441,https://homestuck.com/story/1441,6/003341,NaN,NaN,NaN,NaN,"[Part 1, Act 4, John Egbert, Apple Juice, Capt..."
3897,story,003184,[I] SS: Take a moment to think up some time-ba...,Ok you think you got one. <br /><br /><span st...,1265014234,/storyfiles/hs2/01284.gif,[003185],003183,,[],Homestuck,1284,6/1284,https://homestuck.com/story/1284,6/003184,NaN,NaN,NaN,NaN,"[Part 1, Intermission, Felt Manor, Clock, Bloo..."
3203,story,002628,WV: Curiously prod the funny-looking spirograph.,It appears the funny-looking spirograph room i...,1254620119,/storyfiles/hs2/00728.gif,[002629],002627,,[],Homestuck,728,6/728,https://homestuck.com/story/728,6/002628,NaN,NaN,NaN,1 → 2 → 3,"[Part 1, Act 2, Screen, Sburb Logo, Spyrograph]"
2563,story,002083,John: Captchalogue fanciful harlequins.,You take TWO (2) FANCIFUL HARLEQUINS.<br /><br...,1242785116,/storyfiles/hs2/00183_2.gif,[002084],002082,,[],Homestuck,183,6/183,https://homestuck.com/story/183,6/002083,NaN,NaN,NaN,NaN,"[Act 1, Part 1, Harlequin Doll, Clown, John's ..."
2569,story,002090,John: Captchalogue telescope.,You snatch the TELESCOPE from its TRIPOD. Who ...,1242955053,/storyfiles/hs2/00190_1.gif,[002091],002089,,[],Homestuck,190,6/190,https://homestuck.com/story/190,6/002090,NaN,NaN,NaN,NaN,"[Act 1, Part 1, Earth, John Egbert, Green Slim..."
4270,story,003481,==>,,1268251856,/storyfiles/hs2/01581.gif,[003482],003480,,[],Homestuck,1581,6/1581,https://homestuck.com/story/1581,6/003481,NaN,NaN,NaN,NaN,"[Part 1, Act 4, John Egbert, Wise Guy Slime Su..."
3409,story,002806,Rose: Wear the scarf. Be the Rider.,"Ok, maybe you'll do a LITTLE messing around. Y...",1259011179,/storyfiles/hs2/00906_1.gif,[002807],002805,,[],Homestuck,906,6/906,https://homestuck.com/story/906,6/002806,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Vodka Mutini]"
3327,story,002731,Jade: Open Echidna and go to mspaintadventures...,You open your web browser and visit MSPA.<br /...,1257152576,/storyfiles/hs2/00831.gif,[002732],002730,,[],Homestuck,831,6/831,https://homestuck.com/story/831,6/002731,NaN,NaN,NaN,\nMSPAINT ADVENTURES\n\nGT: Boggle vacantly at...,"[Part 1, Act 3, Homosuck, John Egbert, Toilet,..."
4350,story,003543,Dave: Pester Rose.,|PESTERLOG|<br />-- turntechGodhead <span styl...,1270243098,/storyfiles/hs2/01643_1.gif,[003544],003542,,[],Homestuck,1643,6/1643,https://homestuck.com/story/1643,6/003543,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Dave Strider, Lined Mode]"
3534,story,002909,==>,"Wait, what's that?<br /><br />There's somethin...",1260667088,/storyfiles/hs2/01009.gif,[002910],002908,,[],Homestuck,1009,6/1009,https://homestuck.com/story/1009,6/002909,NaN,NaN,NaN,NaN,"[Part 1, Act 3, WV, Firefly, Serenity, Trusty ..."


# Extracting character appearances from POV cam 

The POV cam extension for Homestuck allows us to see the characters that are present in each page, and not just until page 1988, all of them. We can use this to extract character tags and somewhat make up for the lack of tags in the later pages.

In [414]:
"""
The data from the POV cam comes in many files named after each character, like "roxy.txt" and "rufioh.txt". The format is not meant to be easily parsable, but it shouldn't be too hard to extract the data and "invert" it, to get the characters that appear in each page and their "commands".

An example of the data (jade.txt):
```
Name: Jade
Colour: #4AC925
Image: jade.png
Group: Kids

Be created on meteor
3790-3791
3803
3807
3830-3831

Be sent to Earth
3840

Land on factory
3768-3769

Be adopted
3773-3775

Be taken on hunt with grandfather
Wander off with Bec
Find present
3029-3036
```
From the documentation:
```md
## Timeline language

In the `Readable Timelines` folder are a number of files, each containing the timeline data for a single person.

The files use the following format:

 * Page numbers or ranges of numbers to describe what pages a person's on.
   (For A6A5A1x2 COMBO, use `-2` on the end to go through the pages two at a time)
   eg. `6009`, `1901-2032`, or `7688-7692-2`
 * To split the timeline, indent the splintered timeline, then return to the original indentation for the alpha timeline.
   Note that the two timelines are not connected by default, you must use the next two commands to describe how they should be joined.
 * `==>`: Jump into the next split timeline from this point
 * `<==`: Jump out of previous split timeline to this point
 * `~`: Insert another timeline here, using a person's name.
   eg. `~ Davesprite`

The following commands change properties about the current person or timeline.
Write the exact word, then `:`, then the value you wish to set it to.
eg. `Name: John`.

 * `Name`: Change the name of the current person.
 * `Colour`: Change the colour used for the links.
 * `Image`: Change the image used for the links.
 * `Group`: Change which group the links are a part of.
 * `Caption`: Give some hover-over text to the link between the previous page and the next.

All lines which do not fit any of the above are ignored, like comments.
```
"""


def extract_data_from_pov_cam_file(pov_cam_file: str) -> pd.DataFrame:
    """
    Extracts data from a POV cam file.

    Args:
        pov_cam_file (str): The path to the POV cam file.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data with columns ['page', 'character', 'commands'].
    """
    # Regular expression to identify page lines (single number or range with optional step)
    page_pattern = re.compile(r'^\d+(-\d+)?(-\d+)?$')

    try:
        with open(pov_cam_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"Error: File not found - {pov_cam_file}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])
    except Exception as e:
        print(f"Error reading file {pov_cam_file}: {e}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])

    # Initialize the data storage
    data = {
        'page': [],
        'character': [],
        'commands': []
    }

    # Initialize current character and commands
    current_character = None
    current_commands = []
    last_commands = []

    for line_number, line in enumerate(lines, start=1):
        original_line = line  # Keep the original line for debugging if needed
        line = line.strip()

        if not line:
            continue  # Skip empty lines

        # Check for metadata lines
        if line.startswith("Name: "):
            # Extract character name from the 'Name:' line
            extracted_name = line.split("Name: ", 1)[1].strip()
            if current_character is None:
                current_character = extracted_name
            else:
                # Ignore subsequent 'Name:' lines to prevent name changes from affecting the character name
                continue
            current_commands = []  # Reset commands when a new character is found
            last_commands = []     # Reset last_commands for the new character
            continue  # Move to the next line
        elif any(line.startswith(prefix) for prefix in ["Colour: ", "Image: ", "Group: "]):
            continue  # Skip other metadata lines

        # Check if the line is a page line
        if page_pattern.match(line):
            if current_commands:
                # If there are new commands, update last_commands
                last_commands = current_commands.copy()
                commands_to_use = current_commands.copy()
            elif last_commands:
                # If no new commands, inherit last_commands
                commands_to_use = last_commands.copy()
            else:
                # If no commands to inherit, assign ["==>"]
                commands_to_use = ["==>"]

            # Parse the page or range
            pages = []
            parts = line.split('-')
            try:
                if len(parts) == 1:
                    pages = [int(parts[0])]
                elif len(parts) == 2:
                    start, end = map(int, parts)
                    pages = list(range(start, end + 1))
                elif len(parts) == 3:
                    start, end, step = map(int, parts)
                    pages = list(range(start, end + 1, step))
                else:
                    print(f"Warning: Unexpected page format at line {line_number}: '{original_line}'")
                    continue  # Skip unexpected formats
            except ValueError:
                print(f"Error: Non-integer page numbers at line {line_number}: '{original_line}'")
                continue  # Skip lines with non-integer values

            for page in pages:
                if page in data['page']:
                    # If the page already exists, append the new commands without duplication
                    index = data['page'].index(page)
                    existing_commands = data['commands'][index]
                    for cmd in commands_to_use:
                        if cmd not in existing_commands:
                            existing_commands.append(cmd)
                else:
                    data['page'].append(page)
                    data['character'].append(current_character)
                    data['commands'].append(commands_to_use.copy())  # Use a copy to prevent mutation

            current_commands = []  # Reset commands after assigning them to pages
        else:
            # Assume it's a command line
            current_commands.append(line)

    # Remove duplicate commands per page while preserving order
    for i, cmds in enumerate(data['commands']):
        seen = set()
        unique_cmds = []
        for cmd in cmds:
            if cmd not in seen:
                unique_cmds.append(cmd)
                seen.add(cmd)
        data['commands'][i] = unique_cmds

    # Create DataFrame
    df = pd.DataFrame(data)
    return df


def extract_pov_cam_data(pov_cam_folder: str) -> pd.DataFrame:
    """
    Extracts data from all POV cam files in a given folder.

    Args:
        pov_cam_folder (str): The path to the folder containing POV cam text files.

    Returns:
        pd.DataFrame: A combined DataFrame containing data from all POV cam files.
    """
    # List all .txt files in the folder
    pov_cam_files = [file for file in os.listdir(pov_cam_folder) if file.endswith('.txt')]
    pov_cam_file_with_full_path = [os.path.join(pov_cam_folder, file) for file in pov_cam_files]

    # Initialize a list to hold DataFrames from each file
    pov_cam_data_frames = []

    for file_path in pov_cam_file_with_full_path:
        df = extract_data_from_pov_cam_file(file_path)
        if not df.empty:
            pov_cam_data_frames.append(df)
        else:
            print(f"No data extracted from {file_path}")

    if pov_cam_data_frames:
        pov_cam_data = pd.concat(pov_cam_data_frames, ignore_index=True)
    else:
        pov_cam_data = pd.DataFrame(columns=['page', 'character', 'commands'])

    return pov_cam_data


# Extract data from the POV cam files
pov_cam_data = extract_pov_cam_data(POV_CAM_FOLDER)

# Display the extracted data
pov_cam_data.sample(10)

,page,character,commands
5936,7720,Jack Noir,[Destroy Prospit]
966,7651,Equiusprite,"[Witness arival of tricksters, Witness Fefetas..."
16547,5612,Terezi,"[Find Nepeta's body, Eject disc]"
5371,3819,Grandpa,"[==>, ~ Jake, Arrive on Earth, Kill Colonel Sa..."
5818,9979,Jack Noir,"[Arrive in incipisphere, Land in remains of Pr..."
11163,8124,Kanaya,[Arrive in new session]
4681,4354,Feferi,"[Drink Tab, Disarm.]"
8987,2532,John,"[Shuffle in captchalogue cards, Make pogo hamm..."
7777,6335,Jane,"[Do some sorta lifey thing, Wake up, Talk with..."
16519,5504,Terezi,[Join others in dream bubble]


In [415]:
# Show some unique complex data, like pages with multiple commands
pov_cam_data[pov_cam_data['commands'].apply(len) > 2].sample(10)

,page,character,commands
5141,7376,Gamzee,"[Meet Caliborn, Be shot, repeatedly]"
1532,8800,Caliborn,"[Continue Homosuck, Be interrupted by John, GA..."
13629,3715,Rose,"[Alchemize, Create Thorns of Oglogoth (among o..."
15043,3167,Sawbuck,"[Be interrupted by Snowman, Start jumping arou..."
2733,4937,Dave,"[Caption: Sleeping, Talk with Rose about horro..."
8339,8599,Jane,"[Attempt to resurrect Jade, Be distracted by G..."
940,9812,Equiusprite,"[Finish building houses, Deploy final grist ri..."
10401,9234,John,"[HONK, Be noticed by Gamzee, Honk horn]"
17715,9749,Vriska,"[Call Jake, Request debriefing from Meenah, Fa..."
8240,8346,Jane,"[Glitch out, Interrogate Jake on Derse, Have g..."


In [416]:
# Now that we have the page/character/commands dataset, we can merge the page/character data
# We'll turn the commands into an array of commands prefixed with the character name
# We'll also add a "pov_characters" and "pov_commands" column to the mspa_df
# Note that for the POV cam, the vizPageId is actually incorrect. We can use the actual page number in pageId instead,
# though it's six padded digits in mspa_df_with_tags and an actual integer in pov_cam_data. We'll padd the latter to match.

# Pad the page numbers in pov_cam_data
pov_cam_data['pageId'] = pov_cam_data['page'].apply(lambda x: f"{x:06d}")

# Merging the data now would be silly, we need to create a new dataframe where there's only one row per page
# We'll concatenate the commands for each character in the same page
# We'll also add the commands to a list, and the characters to a list

# Add the character names to the commands if the commands don't already have them
pov_cam_data['commands'] = pov_cam_data.apply(lambda x: [f"{x['character']}: {cmd}" for cmd in x['commands']], axis=1)

# Group by pageId and character
grouped = pov_cam_data.groupby(['pageId']).agg({
    'character': list,
    'commands': list
}).reset_index()

grouped.sample(10)

,pageId,character,commands
861,002768,[Rose],[[Rose: Lose Jaspers]]
2826,004803,[Rose],[[Rose: Use LOWAS as reading light]]
115,002016,[John],"[[John: Be introduced, John: Mess around]]"
3843,005882,"[Jade, Kanaya, Karkat]","[[Jade: Talk with Karkat, Kanaya, Jade: Rememb..."
5578,007736,"[Dirk, Jack Noir, Jane]","[[Dirk: Ascend], [Jack Noir: Destroy Prospit's..."
6973,009193,"[John, Terezi]","[[John: MURD3R, John: Deploy witness at scene ..."
454,002360,[Dave],"[[Dave: Go on Bro's computer, Dave: Talk to Jo..."
5706,007900,"[Aradia, Aranea, Caliborn, Feferi, John, Meena...","[[Aradia: Be in Vriska's crew], [Aranea: Join ..."
7263,009489,"[Jane, Jaspersprite, John, Rose, Rosesprite, R...","[[Jane: Fall through spiral, Jane: Arrive on y..."
6777,008991,"[Calliope, Jade, John]","[[Calliope: Start drawing story, Calliope: Be ..."


In [417]:
# Merge the data into mspa_df_with_tags
# "character" can become "pov_characters" and "commands" can become "pov_commands"

# rename columns
grouped.rename(columns={'character': 'pov_characters', 'commands': 'pov_commands'}, inplace=True)

mspa_df_with_pov = mspa_df_with_tags.merge(grouped, on='pageId', how='left')

# Replace NaN with empty lists for pages without POV cam data
mspa_df_with_pov['pov_characters'] = mspa_df_with_pov['pov_characters'].apply(lambda x: x if isinstance(x, list) else [])
mspa_df_with_pov['pov_commands'] = mspa_df_with_pov['pov_commands'].apply(lambda x: x if isinstance(x, list) else [])

mspa_df_with_pov[mspa_df_with_pov['pov_characters'].apply(len) > 0].sample(10) 

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands
5145,story,004148,======>,,1279247849,/storyfiles/hs2/02248_2.gif,[004149],004147,,[],...,6/2248,https://homestuck.com/story/2248,6/004148,NaN,NaN,NaN,NaN,[],"[Aradia, Sollux, Vriska]","[[Aradia: Be killed], [Sollux: Be manipulated ..."
4500,story,003671,John: Explore lab further.,Now what in the hell is going on in here.,1272338619,/storyfiles/hs2/01771.gif,[003672],003670,,[],...,6/1771,https://homestuck.com/story/1771,6/003671,NaN,NaN,NaN,NaN,"[Part 1, Act 4, John Egbert, Maplehoof, Horse,...",[John],"[[John: Explore., John: Arrive on meteor, John..."
2568,story,002088,John: Use pre-punched card with the alchemiter.,There is no slot for a card anywhere to be fou...,1242878556,/storyfiles/hs2/00188.gif,[002089],002087,,[],...,6/188,https://homestuck.com/story/188,6/002088,NaN,NaN,NaN,NaN,"[Act 1, Part 1, John Egbert, Green Slime Ghost...","[John, Rose]","[[John: Get trapped in room], [Rose: Connect t..."
3310,story,002716,Jade: Press the steak button.,"Ok, well it's a rotary dial so there are no bu...",1256433937,/storyfiles/hs2/00816.gif,[002717],002715,,[],...,6/816,https://homestuck.com/story/816,6/002716,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Jade Harley, Jade's Room, Jade...",[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]"
9858,story,008423,==>,,1378075367,/storyfiles/hs2/06520.gif,[008424],008422,A6A6,[],...,6/6523,https://homestuck.com/story/6523,6/008423,[transcript],,div,"<div class=""log""><section>\n<span style=""color...",[],"[Caliborn, John]","[[Caliborn: Continue Homosuck, Caliborn: Be in..."
4767,story,003858,==>,,1276065590,/storyfiles/hs2/01958.gif,[003859],003857,,[],...,6/1958,https://homestuck.com/story/1958,6/003858,NaN,NaN,NaN,NaN,"[Part 1, Act 4, John Egbert, Hero Mode, Crying...","[Jack Noir, John]","[[Jack Noir: Exchange with PM, gift to John], ..."
4371,story,003560,"Meanwhile, hundreds of pages ago...",You open the package. There is something suspi...,1270428444,/storyfiles/hs2/01660.gif,[003561],003559,,[],...,6/1660,https://homestuck.com/story/1660,6/003560,NaN,NaN,NaN,NaN,"[Part 1, Act 4, John Egbert]",[John],"[[John: Be introduced, John: Mess around]]"
5361,story,004348,Eridan: Bother Feferi.,"|PESTERLOG|<br /><span style=""color: #6a006a"">...",1282018112,/storyfiles/hs2/02447.gif,[004349],004347,,[],...,6/2448,https://homestuck.com/story/2448,6/004348,NaN,NaN,NaN,\nAlert : ♒ \nAlert : ♓ \n,[],"[Eridan, Feferi]","[[Eridan: Do something ridiculous, Eridan: Tal..."
10543,story,009090,[A6A6I4] ====>,,1418792349,/storyfiles/hs2/07187.gif,[009091],009089,,[],...,6/7190,https://homestuck.com/story/7190,6/009090,NaN,NaN,NaN,NaN,[],[Jane],"[[Jane: Follow the yellow brick road, Jane: Fi..."
6625,story,005508,==>,"|PESTERLOG|<br /><span style=""color: #000056"">...",1299115869,/storyfiles/hs2/03605_2.gif,[005509],005507,,[],...,6/3608,https://homestuck.com/story/3608,6/005508,NaN,NaN,NaN,NaN,[],"[Aradia, Jade, Kanaya, Tavros, Terezi]","[[Aradia: ======>], [Jade: Meet Aradia, Aradia..."


# Add Acts and Pages of Homestuck data

I completely forgot this existed until I was already trying to finetune a model, but this is critical information. For each act and even groups of pages, we have detailed synopses. We'll actually need to crawl and generate the dataset ourselves.

In [476]:
"""
```html
<div>
<link rel="stylesheet" type="text/css" href="style-nav.css">

<table id="sidebar-nav">
<tbody><tr>
<th colspan="4" style="font-size: larger;">
<p><a href="/homestuck/"><img alt="Main Page" src="/homestuck/sburb-in-spiro.png" height="200"></a></p>
<p style="margin-top: 0.3em; font-size: 80%;">the Acts and Pages of</p>
<p style="margin-top: 0.1em; font-size: 140%;"><a href="https://homestuck.com/story/">HOMESTUCK</a></p>
<p style="margin: 0.5em 0.2em; font-size: 90%; line-height: 0.95em;">a guide for fans of<br>the web adventure</p>
</th>
</tr>
<tr></tr>
<tr>
<td><strong><img src="/homestuck/map/a1.png" alt=""><br>Act 1</strong></td>
<td><a href="/homestuck/act-2"><img src="/homestuck/map/a2.png" alt=""><br>Act 2</a></td>
<td><a href="/homestuck/act-3"><img src="/homestuck/map/a3.png" alt=""><br>Act 3</a></td>
<td><a href="/homestuck/intermission"><img src="/homestuck/map/int.png" alt=""><br>Int</a></td>
</tr>
<tr>
<td><a href="/homestuck/act-4"><img src="/homestuck/map/a4.png" alt=""><br>Act 4</a></td>
<td><a href="/homestuck/a5a1"><img src="/homestuck/map/a5a1.png" alt=""><br>5–1</a></td>
<td><a href="/homestuck/a5a2"><img src="/homestuck/map/a5a2.png" alt=""><br>5–2</a></td>
<td class="scaps"><a href="/homestuck/eoa5"><img src="/homestuck/map/cascade.png" alt=""><br>EOA5</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-1"><img src="/homestuck/map/a6a1.png" alt=""><br>6–1</a></td>
<td><a href="/homestuck/a6-2"><img src="/homestuck/map/a6a2.png" alt=""><br>6–2</a></td>
<td><a href="/homestuck/a6-3"><img src="/homestuck/map/a6a3.png" alt=""><br>6–3</a></td>
<td><a href="/homestuck/a6-4"><img src="/homestuck/map/a6a4.png" alt=""><br>6–4</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-5"><img src="/homestuck/map/a6a5.png" alt=""><br>6–5</a></td>
<td><a href="/homestuck/a6a6-1"><img src="/homestuck/map/a6a6a1-hi.png" alt=""><br>6-6-1</a></td>
<td><a href="/homestuck/a6a6-2"><img src="/homestuck/map/a6a6a2-hi.png" alt=""><br>6-6-2</a></td>
<td><a href="/homestuck/a6a6-3"><img src="/homestuck/map/a6a6a3-hi.png" alt=""><br>6-6-3</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6a6-4"><img src="/homestuck/map/a6a6a4-hi.png" alt=""><br>6-6-4</a></td>
<td><a href="/homestuck/a6a6-5"><img src="/homestuck/map/a6a6a5-hi.png" alt=""><br>6-6-5</a></td>
<td><a href="/homestuck/a6a6a6"><img src="/homestuck/map/a6a6a6-hi.png" alt=""><br>6-6-6</a></td>
<td><a href="/homestuck/act-7"><img src="/homestuck/map/a7.png" alt=""><br>Act 7</a></td>
</tr>
<tr>
<td><a href="/homestuck/part-1"><img src="/homestuck/sburb.png" alt=""><br>Part 1</a></td>
<td><a href="/homestuck/part-2"><img src="/homestuck/sgrub.png" alt=""><br>Part 2</a></td>
<td><a href="/homestuck/part-3"><img src="/homestuck/sburb-a.png" alt=""><br>Part 3</a></td>
<td><a href="/homestuck/part-4"><img src="/homestuck/sburb-r-small.png" alt=""><br>Part 4</a></td>
</tr>
<tr></tr>
<tr>
<th colspan="4">
<p style="font-size: 120%; line-height: 1.1em; margin-top: 0.2em;">documented by</p>
<p style="font-size: 140%; line-height: 1em;">Rafe Saltman</p>
<p style="font-size: 80%;"><a href="mailto:rafe@rafe.name">email</a> &nbsp;<a href="https://rafr.tumblr.com">tumblr</a>&nbsp; <a href="https://twitter.com/Rafes">twitter</a></p>
<p style="font-size: 120%; line-height: 1em; margin-top: 1.2em;">art and excerpts<br>from <i>Homestuck</i></p>
<p style="font-size: 120%; line-height: 1.5em;"><img src="/homestuck/mspa-face.png" class="face"> © 2009-2016 <img src="/homestuck/mspa-flip.png" class="face"></p>
<p style="font-size: 140%">Andrew Hussie</p>
</th>
</tr>
</tbody></table>

<table id="header-nav">
<tbody><tr>
<th rowspan="6">
<p><a href="/homestuck/"><img alt="Main Page" src="/homestuck/sburb-in-spiro.png" height="175"></a></p>
<p style="margin-top: 0.6em; font-size: 85%;">the Acts and Pages of</p>
<p style="font-size: 150%;"><a href="https://homestuck.com/story/">HOMESTUCK</a></p>
<p style="margin-top: 0.5em; font-size: 95%; line-height: 1em;">a guide for fans of<br>the web adventure</p>
<p style="line-height: 1.1em; margin-top: 1.5em;">documented by</p>
<p style="font-size: 115%;">Rafe Saltman</p>
<p style="font-size: 72%;"><a href="mailto:rafe@rafe.name">email</a>&nbsp; <a href="https://rafr.tumblr.com">tumblr</a> &nbsp;<a href="https://twitter.com/Rafes">twitter</a></p>
<p style="line-height: 1.1em; margin-top: 1.5em;">art and excerpts<br>from <i>Homestuck</i></p>
<p><img src="/homestuck/mspa-face.png" class="face"> © 2009-2016 <img src="/homestuck/mspa-flip.png" class="face"></p>
<p style="font-size: 115%;">Andrew Hussie</p>
</th>
<td><strong><img src="/homestuck/map/a1.png" alt=""><br>Act 1</strong></td>
<td><a href="/homestuck/act-2"><img src="/homestuck/map/a2.png" alt=""><br>Act 2</a></td>
<td><a href="/homestuck/act-3"><img src="/homestuck/map/a3.png" alt=""><br>Act 3</a></td>
<td><a href="/homestuck/intermission"><img src="/homestuck/map/int.png" alt=""><br>Int</a></td>
</tr>
<tr>
<td><a href="/homestuck/act-4"><img src="/homestuck/map/a4.png" alt=""><br>Act 4</a></td>
<td><a href="/homestuck/a5a1"><img src="/homestuck/map/a5a1.png" alt=""><br>5–1</a></td>
<td><a href="/homestuck/a5a2"><img src="/homestuck/map/a5a2.png" alt=""><br>5–2</a></td>
<td class="scaps"><a href="/homestuck/eoa5"><img src="/homestuck/map/cascade.png" alt=""><br>EOA5</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-1"><img src="/homestuck/map/a6a1.png" alt=""><br>6–1</a></td>
<td><a href="/homestuck/a6-2"><img src="/homestuck/map/a6a2.png" alt=""><br>6–2</a></td>
<td><a href="/homestuck/a6-3"><img src="/homestuck/map/a6a3.png" alt=""><br>6–3</a></td>
<td><a href="/homestuck/a6-4"><img src="/homestuck/map/a6a4.png" alt=""><br>6–4</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-5"><img src="/homestuck/map/a6a5.png" alt=""><br>6–5</a></td>
<td><a href="/homestuck/a6a6-1"><img src="/homestuck/map/a6a6a1-hi.png" alt=""><br>6-6-1</a></td>
<td><a href="/homestuck/a6a6-2"><img src="/homestuck/map/a6a6a2-hi.png" alt=""><br>6-6-2</a></td>
<td><a href="/homestuck/a6a6-3"><img src="/homestuck/map/a6a6a3-hi.png" alt=""><br>6-6-3</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6a6-4"><img src="/homestuck/map/a6a6a4-hi.png" alt=""><br>6-6-4</a></td>
<td><a href="/homestuck/a6a6-5"><img src="/homestuck/map/a6a6a5-hi.png" alt=""><br>6-6-5</a></td>
<td><a href="/homestuck/a6a6a6"><img src="/homestuck/map/a6a6a6-hi.png" alt=""><br>6-6-6</a></td>
<td><a href="/homestuck/act-7"><img src="/homestuck/map/a7.png" alt=""><br>Act 7</a></td>
</tr>
<tr>

<td style="vertical-align: bottom;"><a href="/homestuck/part-1"><img src="/homestuck/sburb.png" alt=""><br>Part 1</a></td>
<td style="vertical-align: bottom;"><a href="/homestuck/part-2"><img src="/homestuck/sgrub.png" alt=""><br>Part 2</a></td>
<td style="vertical-align: bottom;"><a href="/homestuck/part-3"><img src="/homestuck/sburb-a.png" alt=""><br>Part 3</a></td>
<td style="vertical-align: bottom;"><a href="/homestuck/part-4"><img src="/homestuck/sburb-r-small.png" alt=""><br>Part 4</a></td>
</tr>
</tbody></table>
<table>
<tbody><tr>
<th style="width: 160px"><a href="#a1-0"><img src="map/a1-hi.png" alt="Act 1 thumbnail" height="148px"></a></th>
<th colspan="3">
<h1 class="tint">Act 1</h1>
<h1 class="title">The Note Desolation Plays</h1>
<h4 class="spoilers">Spoilers ahead!</h4></th>
</tr>
<tr class="stats">
<th>first page</th>
<td><a href="/hs/1">1</a>, April 13 2009</td>
<th>pages</th>
<td>247</td>
</tr>
<tr class="stats">
<th>last page</th>
<td><a href="/hs/247">247</a>, June 7 2009</td>
<th>words in text</th>
<td id="words-text">10 563</td>
</tr>
<tr class="stats">
<th>recap</th>
<td><a href="/hs/1674">1674</a>, April 15 2010</td>
<th><a href="http://readmspa.org/search/">words in media</a></th>
<td id="words-media">2032</td>
</tr>
<tr><th></th><th colspan="3" style="padding-top: 1em;">Synopsis</th></tr>
<tr class="synopsis"><th>john</th>
<td colspan="3">
<p>Homestuck begins as the story of four friends who live far apart and have never met. We are introduced to these kids one by one, starting with John Egbert.</p>
<p>It is John’s thirteenth birthday, and he wants to play a game with his online friends. He has been waiting for the game to arrive in the mail for days. After fussing with a gift from his baking-crazed Dad and enduring the pesterings of his friends, John leaves his room to check the mailbox.</p>
<p>Seeing that Dad has already brought in the mail, John confronts him in the kitchen. They have a messy battle with hammers and cakes and pies. John ends the strife with a distraction and makes off with his mail.</p>
<p>There are two parcels addressed to himself. The first, a gift from his friend Dave, contains the plush bunny prop from the set of <i>Con Air</i>. The second contains the beta release of the computer game Sburb. John and his friend Rose are eager to play Sburb and find out what it is about.</p>
<p>Two other parcels for John remain out of reach, locked in Dad’s car.</p>
</td></tr>
<tr class="synopsis"><th>sburb</th>
<td colspan="3">
<p>John starts a game of Sburb with Rose.</p>
<p>As John’s server player, Rose assumes control of his environment. Experimenting with the controls on her end, Rose discovers that she can extend John’s house, move furniture around, and place machinery that supports a kind of alchemy based on punch cards.</p>
<p>The alchemy equipment emits a confusing ball of light called the Kernelsprite. The sprite is “prototyped” when it absorbs John’s maimed harlequin doll and takes on the appearance of a one-eyed, one-armed jester.</p>
<p>The mission to alchemize their first item becomes urgent when John discovers a meteor is about to hit his house. After a series of frustrations brought on by Rose’s spotty internet connection, they succeed in creating a tree that bears a blue apple. John has only enough time to take a bite of the apple before the meteor strikes the Earth and makes a crater where his house stood a moment before.</p>
</td></tr>
</tbody></table>
<table>
<tbody><tr>
<th colspan="4">Act 1 by date posted</th>
</tr>
<tr></tr>
<tr>
<th>date</th><th>page</th><th>scenes and events</th><th>adventure log</th>
</tr>
<tr id="a1-0">
<td>April 13 2009</td>
<td>1</td>
<td>
<p><img src="map/a1.png" height="74" alt=""></p>
<p><strong class="caps">Act 1.</strong></p>
<p>Who is this young man?</p>
<p>John Egbert examines his room.</p>
</td>
<td class="log">
<p id="1"><a href="/hs/1">Homestuck</a></p>
<p id="2"><a href="/hs/2">Enter name.</a></p>
<p id="3"><a href="/hs/3">Try again.</a></p>
<p id="4"><a href="/hs/4">Examine room.</a></p>
<p id="5"><a href="/hs/5">John: Quickly retrieve arms from drawer.</a></p>
<p id="6"><a href="/hs/6">Remove CAKE from MAGIC CHEST.</a></p>
<p id="7"><a href="/hs/7">John: Quickly retrieve arms from MAGIC CHEST.</a></p>
<p id="8"><a href="/hs/8">John: Examine contents of chest.</a></p>
<p id="9"><a href="/hs/9">John: Captchalogue smoke pellets.</a></p>
<p id="10"><a href="/hs/10">John: Equip fake arms.</a></p>
<p id="11"><a href="/hs/11">John: Examine Problem Sleuth Poster.</a></p>
<p id="12"><a href="/hs/12">John: Read note on drawer.</a></p>
<p id="13"><a href="/hs/13">John: Take poster.</a></p>
<p id="14"><a href="/hs/14">John: Acquire hammer and nails. They will come in handy.</a></p>
<p id="15"><a href="/hs/15">John: Take nails.</a></p>
<p id="16"><a href="/hs/16">John: Squawk like an imbecile and shit on your desk.</a></p>
</td>
</tr>
<tr>
<td>April 14 2009</td>
<td>17</td>
<td>
<p>John puts up his new Little Monsters poster.</p>
</td>
<td class="log">
<p id="17"><a href="/hs/17">John: Combine the nails and hammer.</a></p>
<p id="18"><a href="/hs/18">John: Use hammer/nails on poster.</a></p>
<p id="19"><a href="/hs/19">John: Nail poster to wall.</a></p>
<p id="20"><a href="/hs/20">John: Examine Con Air poster.</a></p>
<p id="21"><a href="/hs/21">John: Examine Deep Impact poster.</a></p>
<p id="22"><a href="/hs/22">John: Examine calendar.</a></p>
<p id="23"><a href="/hs/23">John: Eat cake.</a></p>
</td>
</tr>
...
```

The above is an excerpt of the HTML in one of the pages of the website, specifically https://rafe.name/homestuck/act-1.
We'll obviously have to crawl all the act pages, and the part pages. Each part contains multiple acts, each act contains multiple groups of pages. 
We'll end up with a dataset (after exploding the group-based dataset) like...
page_id | page_group_summary | act | act_summary | part | part_summary
1 | "John Egbert examines his room." | "Act 1" | [["JOHN", "Homestuck begins as the story of four friends who live far apart and have never met. We are introduced to these kids one by one, starting with John Egbert...."], ["SBURB", "The alchemy equipment emits a confusing ball of light called the Kernelsprite. The sprite is “prototyped” when it absorbs John’s maimed harlequin doll and takes on the appearance of a one-eyed, one-armed jester..."]] | "Part 1" | "John, Rose, Dave, and Jade are online friends. On John’s thirteenth birthday, the kids play a game that takes them and their guardians to a set of fantastic worlds called the Medium. They are challenged to work together, kill monsters, learn punch-card alchemy, and build their houses up to the heavenly orb of Skaia..."

This should be really useful when we're trying to guess the tags that are in a page. The summaries should also be generally useful for text-based models later on. 
"""




'\n```html\n<div>\n<link rel="stylesheet" type="text/css" href="style-nav.css">\n\n<table id="sidebar-nav">\n<tbody><tr>\n<th colspan="4" style="font-size: larger;">\n<p><a href="/homestuck/"><img alt="Main Page" src="/homestuck/sburb-in-spiro.png" height="200"></a></p>\n<p style="margin-top: 0.3em; font-size: 80%;">the Acts and Pages of</p>\n<p style="margin-top: 0.1em; font-size: 140%;"><a href="https://homestuck.com/story/">HOMESTUCK</a></p>\n<p style="margin: 0.5em 0.2em; font-size: 90%; line-height: 0.95em;">a guide for fans of<br>the web adventure</p>\n</th>\n</tr>\n<tr></tr>\n<tr>\n<td><strong><img src="/homestuck/map/a1.png" alt=""><br>Act 1</strong></td>\n<td><a href="/homestuck/act-2"><img src="/homestuck/map/a2.png" alt=""><br>Act 2</a></td>\n<td><a href="/homestuck/act-3"><img src="/homestuck/map/a3.png" alt=""><br>Act 3</a></td>\n<td><a href="/homestuck/intermission"><img src="/homestuck/map/int.png" alt=""><br>Int</a></td>\n</tr>\n<tr>\n<td><a href="/homestuck/act-4"><img

# Cleaning up

We plan on converting this to CSV and potentially JSONL, so right now have a lot of object columns that should be strings, a bunch of empty strings and NaNs that should be pd.NAs. 

In [418]:
# mspa_df_with_pov currently has these columns representing MSPA data:
# section (str, per page): the section of the MSPA website the page is in, always a string like "story"
# pageId (str, per page): the page number, usually a six digit padded string like "001901", sometimes a regular string like "pony"
# title (str, per page): the title of the page, usually the command given in the previous page, always a string like "John: Enter name."
# content (str, per page): the content of the page, usually the page minus the panels (includes dialog and HTML)
# timestamp (timestamp, per page): the UNIX timestamp of the page, always a number like 1580000000
# src (str, per panel): the partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like "/storyfiles/hs2/001901.gif"
# next (list of strings, per page): the pageId of the next pages, usually a list of one string like ["001902"]
# previous (string, per page): the pageId of the previous page, usually a string like "001900"
# theme (str, per page): the theme of the page, like "A6A6", always a string, usually empty
# flag (list of strings, per page): a list of flags for Homestuck Collection presentation, some flags like ["R1"] show that the page is part of a "retcon" where Hussie went back and edited panels, others like ["FIREFLY"] show the page has special javascript that shows fireflies flying around the page, a page usually has no flag
# vizStory (str, per page): essentially a readable version of the section, always a string like "Homestuck"
# vizPageNumber (str, per page): pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ, always a non-padded string since string pages like "pony" still exist
# vizPageId (str, per page): we should remove this column, being a combination of section number and viz page that was only used in one dataset we wanted to merge with
# vizLink (str, per page): the link to the page in the VIZ website, always a full URI like "https://homestuck.com/story/5320"
# pageNumber (str, per page): like vizPageId but for MSPA format, also should be removed
# transcript class (str, per panel): the class of the transcript, like "dialogue" or "narration", always a string, usually a NaN which should be replaced with None
# transcript style (str, per panel): the style of the transcript, in CSS, like "font-weight: bold", always a string, usually a NaN which should be replaced with None
# transcript tag (str, per panel): the tag of the transcript, like "p" or "div", always a string, usually a NaN which should be replaced with None
# transcript (str, per panel): the transcript of the panel, the text that appears in the panel, always a string, usually a NaN which should be replaced with None
# tags (list of strings, per page): the tags of the page, like ["John Egbert", "Rose Lalonde", "Land of Wind and Shade"], always a list of strings, usually an empty list
# pov_characters (list of strings, per page): the characters that appear in the page according to the POV cam, always a list of strings, usually an empty list
# pov_commands (list of strings, per page): the commands that the characters have in the page according to the POV cam, always a list of lists of strings (per character), usually an empty list

# Let's start by removing the columns we don't need
mspa_df_cleaned = mspa_df_with_pov.drop(columns=['vizPageId', 'pageNumber'])

# Replace NaN with pd.NA in the transcript columns
na_to_pdna_columns = ['transcript_class', 'transcript_style', 'transcript_tag', 'transcript']
mspa_df_cleaned[na_to_pdna_columns] = mspa_df_cleaned[na_to_pdna_columns].where(mspa_df_cleaned[na_to_pdna_columns].notnull(), pd.NA)

# Replace empty strings with pd.NA in all columns
mspa_df_cleaned = mspa_df_cleaned.replace('', pd.NA)
        
# Show some cleaned data
mspa_df_cleaned.sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizLink,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands
618,story,000483,Next.,Some icons beneath the plaques are revealed.,1209080354,/advimgs/ps/ps265.gif,[000484],000482,retro,[],Problem Sleuth,265,https://homestuck.com/problem-sleuth/265,<NA>,<NA>,<NA>,<NA>,[],[],[]
3750,story,003060,[I] ==>,You cover the unsightly individual back up and...,1263671776,/storyfiles/hs2/01160.gif,[003061],003059,<NA>,[],Homestuck,1160,https://homestuck.com/story/1160,<NA>,<NA>,<NA>,<NA>,"[Part 1, Intermission, Felt Manor, Stairs, Mid...",[Spades Slick],"[[Spades Slick: Enter mansion, Spades Slick: C..."
3617,story,002959,"John: Combine fake arm, blue ectoplasm, and PDA.",You make the REMOTE GHOST GAUNTLET. <br /><br ...,1261462047,/storyfiles/hs2/01059_3.gif,[002960],002958,<NA>,[],Homestuck,1059,https://homestuck.com/story/1059,<NA>,<NA>,<NA>,<NA>,"[Part 1, Act 3, John Egbert, Wise Guy Slime Su...",[John],[[John: Alchemise]]
2085,story,001717,Next.,"You consume your counterparts, ZAD and FAD, fi...",1235189629,/advimgs/ps/ps1504_2.gif,[001718],001716,retro,[],Problem Sleuth,1499,https://homestuck.com/problem-sleuth/1499,<NA>,<NA>,<NA>,<NA>,[],[],[]
3691,story,003016,PM : Read the letter.,This is kind of confusing.,1262581972,/storyfiles/hs2/01116.gif,[003017],003015,<NA>,[],Homestuck,1116,https://homestuck.com/story/1116,<NA>,<NA>,<NA>,<NA>,"[Part 1, Act 3, Lined Mode, Carapacian, Exile,...","[Parcel Mistress, Warweary Villein]","[[Parcel Mistress: Read letter from Jade, Parc..."
380,story,000037,BARF!,Overcome with nausea from the foul chain of ev...,1180933316,/advimgs/jb/mspaintadventure32.gif,[000038],000036,retro,[],Jailbreak,37,https://homestuck.com/jailbreak/37,<NA>,<NA>,<NA>,BARF,[],[],[]
5608,story,004587,==>,<NA>,1285724961,/storyfiles/hs2/02686.gif,[004588],004586,<NA>,[],Homestuck,2687,https://homestuck.com/story/2687,<NA>,<NA>,<NA>,<NA>,[],[Rose],[[Rose: Talk with John about building his hous...
6145,story,005074,==>,All of your computers light up at once as some...,1293505366,/storyfiles/hs2/03171.gif,[005075],005073,<NA>,[],Homestuck,3174,https://homestuck.com/story/3174,<NA>,<NA>,<NA>,Alert : ♒ x 7,[],"[Eridan, Jade]","[[Eridan: Troll Jade], [Jade: Prepare for Chri..."
7944,story,006673,[A6I2] ==>,"|PESTERLOG|<br /><span style=""color: #008282"">...",1333016180,/storyfiles/hs2/04770.gif,[006674],006672,<NA>,[],Homestuck,4773,https://homestuck.com/story/4773,<NA>,<NA>,<NA>,<NA>,[],"[Aranea, Terezi]","[[Aranea: Talk to Terezi, Aranea: Pretend to b..."
389,story,000283,Next.,"It is your loathesome arch nemesis, Ace Dick! ...",1205618713,/advimgs/ps/ps065.gif,[000284],000282,retro,[],Problem Sleuth,65,https://homestuck.com/problem-sleuth/65,<NA>,<NA>,<NA>,<NA>,[],[],[]


In [419]:
# Fix the inconsistent casing (some columns are snake_case, others camelCase)
# Annoyingly, there doesn't seem to be a standard for this for ML datasets, so we'll go with Python's snake_case
# This means altering pageId, vizStory, vizPageNumber, vizLink

mspa_df_cleaned.rename(columns={
    'pageId': 'page_id',
    'vizStory': 'viz_story',
    'vizPageNumber': 'viz_page_number',
    'vizLink': 'viz_link'
}, inplace=True)

mspa_df_cleaned.sample(5)

,section,page_id,title,content,timestamp,src,next,previous,theme,flag,viz_story,viz_page_number,viz_link,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands
2405,story,001954,John: Combine father's pipe with clever disguise.,You think now would be a good time to beef up ...,1240094060,/storyfiles/hs2/00054.gif,[001955],001953,<NA>,[],Homestuck,54,https://homestuck.com/story/54,<NA>,<NA>,<NA>,<NA>,"[Part 1, Act 1, John's House, Living Room, Joh...",[John],"[[John: Be introduced, John: Mess around]]"
6481,story,005388,==>,"|PESTERLOG|<br /><span style=""color: #e00707"">...",1297473773,/storyfiles/hs2/03485.gif,[005389],005387,<NA>,[],Homestuck,3488,https://homestuck.com/story/3488,<NA>,<NA>,<NA>,<NA>,[],"[Aradia, Dave]","[[Aradia: Visit coinflip Dave], [Dave: Be in d..."
1121,story,000934,Next.,The GRENADES detonate. All of your WAFERS have...,1220241352,/advimgs/ps/ps722.gif,[000935],000933,retro,[],Problem Sleuth,716,https://homestuck.com/problem-sleuth/716,<NA>,<NA>,<NA>,\nGRIT CIBORIUM\n\nBOO- BOOM\n,[],[],[]
6213,story,005136,AH: Apply duds.,"Awesome, works for me.",1294014574,/storyfiles/hs2/03233.gif,[005137],005135,<NA>,[],Homestuck,3236,https://homestuck.com/story/3236,<NA>,<NA>,<NA>,<NA>,[],[],[]
10184,story,008745,[A6A6I2] ====>,<NA>,1381908556,/storyfiles/hs2/06842.gif,[008746],008744,<NA>,[],Homestuck,6845,https://homestuck.com/story/6845,<NA>,<NA>,<NA>,<NA>,[],"[Aranea, Jane]","[[Aranea: The Condesce arrives], [Jane: Condes..."


In [420]:
# Make column types explicit as verification and so they don't resolve to float when we save them, and timestamp stays a timestamp

mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')

# Convert all columns to string except for the lists and timestamp

for column in mspa_df_cleaned.columns:
    if column not in ['timestamp', 'next', 'tags', 'pov_characters', 'pov_commands', 'flag']:
        mspa_df_cleaned[column] = mspa_df_cleaned[column].astype('string')
        
mspa_df_cleaned.sample(5)

C:\Users\PC\AppData\Local\Temp\ipykernel_2528\1107193909.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')


,section,page_id,title,content,timestamp,src,next,previous,theme,flag,viz_story,viz_page_number,viz_link,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands
2135,story,001755,Weasel King: Commune with arbitor.,The souls of the ARBITOR and the KING merge as...,2009-03-01 06:35:08,/advimgs/ps/ps1541.gif,[001756],001754,retro,[],Problem Sleuth,1537,https://homestuck.com/problem-sleuth/1537,<NA>,<NA>,<NA>,<NA>,[],[],[]
9619,story,008196,[A6A6I1] ====>,"|DIALOGLOG|<br /><span style=""color: #ff6ff2"">...",2013-06-18 02:41:41,/storyfiles/hs2/06294.gif,[008197],008195,<NA>,[],Homestuck,6296,https://homestuck.com/story/6296,<NA>,<NA>,<NA>,<NA>,[],"[Jade, Roxy]","[[Jade: Intimidate Roxy, Jade: Tell Roxy about..."
5453,story,004430,To begin your mission.,<NA>,2010-08-26 01:21:40,/storyfiles/hs2/02529.gif,[004431],004429,<NA>,[],Homestuck,2530,https://homestuck.com/story/2530,<NA>,<NA>,<NA>,TEREZI: WH4T M1SS1ON >:?,[],"[Snowman, Terezi]","[[Snowman: Be exile for Vriska, Terezi], [Tere..."
5403,story,004388,======>,Ugh. This troll paint is making a mess. This w...,2010-08-22 00:43:07,/storyfiles/hs2/02487.gif,[004389],004387,<NA>,[],Homestuck,2488,https://homestuck.com/story/2488,<NA>,<NA>,<NA>,<NA>,[],[],[]
10241,story,008794,...........,<NA>,2014-10-19 05:42:35,/storyfiles/hs2/06891_1.gif,[008795],008793,A6A6,[],Homestuck,6894,https://homestuck.com/story/6894,<NA>,<NA>,<NA>,ANOTHER INTENSE STAREDOWN,[],"[Caliborn, John]","[[Caliborn: Continue Homosuck, Caliborn: Be in..."


In [421]:
# Let's reorder the columns to make it easier to read, the first columns should let a user know the story and the title of the page at least

mspa_df_cleaned = mspa_df_cleaned[['viz_story', 'title', 'content', 'page_id', 'timestamp', 'src', 'next', 'previous', 'viz_page_number', 'viz_link', 'tags', 'pov_characters', 'pov_commands', 'transcript_class', 'transcript_style', 'transcript_tag', 'transcript', 'theme', 'flag']]

mspa_df_cleaned.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,viz_page_number,viz_link,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,theme,flag
9939,Homestuck,[A6A6I2] ====>,"|SPRITELOG|<br /><span style=""color: #e00707"">...",008505,2013-09-09 18:19:45,/storyfiles/hs2/06602.gif,[008506],008504,6605,https://homestuck.com/story/6605,[],"[Equiusprite, Davesprite, Erisolsprite, Jasper...",[[Equiusprite: It is exceeding a certain amoun...,<NA>,<NA>,<NA>,<NA>,<NA>,[]
210,Jailbreak,Drill through the floor,For the first time in your 3 year imprisonment...,000047,2007-06-04 05:18:08,/advimgs/jb/mspaintadventure40e.gif,[000048],000046,47,https://homestuck.com/jailbreak/47,[],[],[],<NA>,<NA>,<NA>,WHIIIIR!,retro,[]
9470,Homestuck,[A6I5] ==>,<NA>,008055,2013-04-10 01:34:04,/storyfiles/hs2/06152.gif,[008056],008054,6155,https://homestuck.com/story/6155,[],[John],"[[John: Jump around space, time]]",<NA>,<NA>,<NA>,zap.,<NA>,[]
4575,Homestuck,Dave: Combine skateboard and Hella Jeff drawing.,You make UNREAL AIR.,003729,2010-05-13 16:08:23,/storyfiles/hs2/01829_1.gif,[003730],003728,1829,https://homestuck.com/story/1829,"[Part 1, Act 4, Dave Strider, Alchemy, Suit, R...",[Dave],[[Dave: Wake up and jam.]],<NA>,<NA>,<NA>,UNREAL AIR ⬡ 0,<NA>,[]
8772,Homestuck,==>,No surprise here. The archagent is dominating ...,007460,2012-12-10 03:19:42,/storyfiles/hs2/05557.gif,[007461],007459,5560,https://homestuck.com/story/5560,[],[Jack Noir],[[Jack Noir: Continue being imprisoned]],<NA>,<NA>,<NA>,The Enquiring Carapacian JAILGATE Day 153 D...,<NA>,[]


In [422]:
# And let's make a dict with column descriptions for later when we release the dataset

dataset_column_descriptions = {
    'viz_story': "The story the page is part of, always 'Homestuck'.",
    'title': "The title of the page, usually the command given in the previous page.",
    'content': "The content of the page, usually the page minus the panels (includes dialog and HTML).",
    'page_id': "The page number, usually a six digit padded string like '001901', sometimes a regular string like 'pony'.",
    'timestamp': "The UNIX timestamp of the page.",
    'src': "The partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like '/storyfiles/hs2/001901.gif'.",
    'next': "The pageId of the next pages, usually a list of one string like ['001902'].",
    'previous': "The pageId of the previous page, usually a string like '001900'.",
    'viz_page_number': "Pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ.",
    'viz_link': "The link to the page in the VIZ website.",
    'tags': "The tags of the page, like ['John Egbert', 'Rose Lalonde', 'Land of Wind and Shade'].",
    'pov_characters': "The characters that appear in the page according to the POV cam.",
    'pov_commands': "The commands that the characters have in the page according to the POV cam.",
    'transcript_class': "The class of the transcript, like 'dialogue' or 'narration'.",
    'transcript_style': "The style of the transcript, in CSS.",
    'transcript_tag': "The tag of the transcript, like 'p' or 'div'.",
    'transcript': "The transcript of the panel, the text that appears in the panel.",
    'theme': "The theme of the page, like 'A6A6'.",
    'flag': "A list of flags for Homestuck Collection presentation."
}

In [423]:
# Before we continue we need to confirm we can fetch the image or asset of each panel
# Pick an asset with a src, like "/storyfiles/hs2/001901.gif"

# We'll use the requests library to fetch the image
import requests

# We just need to display the image in a Jupyter notebook so we'll use IPython's display
from IPython.display import Image

# We'll also use BytesIO to convert the image to bytes
from io import BytesIO

# We'll use a random homestuck panel with enough metadata, like the one in viz page 2519
panel_src = mspa_df_cleaned[mspa_df_cleaned['viz_page_number'] == '2519']['src'].values[0]
panel_url = f"https://www.homestuck.com/images{panel_src}"

# Test the image URL
print(f"Fetching image from {panel_url}")
requests.get(panel_url)


Fetching image from https://www.homestuck.com/images/storyfiles/hs2/02518.gif


<Response [200]>

In [424]:
Image(url=panel_url)

In [425]:
# Right now we haven't even saved the data, so let's do that first, as csv

mspa_df_cleaned.to_csv('mspa_dataset.csv', index=False)

# Now we can load the data back (CSV does have the limitation of not representing pd.NA correctly though, we'll just get NaNs)
mspa_df_csv = pd.read_csv('mspa_dataset.csv')
mspa_df_csv.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,viz_page_number,viz_link,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,theme,flag
6143,Homestuck,Jade: Combine Dutton and sooth specs.,You make the DUTTON BUBBLE GOGGLES!,005071,2010-12-25 07:21:51,/storyfiles/hs2/03168_2.gif,['005072'],5070.0,3171,https://homestuck.com/story/3171,[],['Jade'],"[['Jade: Prepare for Christmas', 'Jade: Be giv...",NaN,NaN,NaN,NaN,NaN,[]
5234,Homestuck,Kanaya: Equip chainsaw.,What CHAINSAW? You are quite sure there is no ...,004224,2010-07-31 22:49:51,/storyfiles/hs2/02324.gif,['004225'],4223.0,2324,https://homestuck.com/story/2324,[],['Kanaya'],"[['Kanaya: Be introduced', 'Kanaya: Talk with ...",NaN,NaN,NaN,NaN,NaN,[]
6440,Homestuck,Nepeta: Pounce.,NaN,005350,2011-02-07 19:37:26,/storyfiles/hs2/03447.gif,['005351'],5349.0,3450,https://homestuck.com/story/3450,[],"['Gamzee', 'Nepeta']","[['Gamzee: Kill Equius', 'Gamzee: Engage murde...",NaN,NaN,NaN,\nhisssssssssssss!\n\nBIMP\n,NaN,[]
544,Problem Sleuth,AD: Use rope to escape office through window.,You won't need to use the ROPE to escape throu...,000409,2008-04-07 19:31:33,/advimgs/ps/ps191.gif,['000410'],408.0,191,https://homestuck.com/problem-sleuth/191,[],[],[],NaN,NaN,NaN,80%,retro,[]
3611,Homestuck,John: Combine gushers and blue ectoplasm.,You mix your Gushers with some of the blue sli...,002958,2009-12-22 06:06:36,/storyfiles/hs2/01058_1.gif,['002959'],2957.0,1058,https://homestuck.com/story/1058,"['Part 1', 'Act 3', 'Gushers', 'Nannasprite']",['John'],[['John: Alchemise']],NaN,NaN,NaN,\nHELLACIOUS BLUE PHLEGM ANEURYSM GUSHERS\n⬡ 2...,NaN,[]


# Tagging panels

We have the panel images, we have the ReadMSPA transcripts of them, we have the point of view from the POV cam, and we have the title and text that accompanies the panels. We have some partial tagging information from the Homestuck Search engine. With that and a vision model, we might be able to successfully extract non-hallucinated information.
Basically we need to predict the tag-related columns, since we only have tagging data for the first four acts.

In [434]:
# First we need to get only the image we need. We can assemble the image URLs from the src column, by prefixing "https://homestuck.com/images" to it
# That, plus the rest of informational columns, will be useful information to train the model with
# We want to predict the "tags" column given the rest, so we take the Act 1-4 data (basically everything that has a non-empty tags column):

four_acts_hs_df = mspa_df_cleaned[mspa_df_cleaned['tags'].apply(len) > 0]
four_acts_hs_df.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,viz_page_number,viz_link,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,theme,flag
2856,Homestuck,==>,"|PESTERLOG|<br /><span style=""color: #4ac925"">...",002342,2009-08-05 23:50:53,/storyfiles/hs2/00442_2.gif,[002343],002341,442,https://homestuck.com/story/442,"[Act 2, Part 1, Rose's Room, Rose's House, Bed...",[Rose],[[Rose: Talk with Jade]],<NA>,<NA>,<NA>,<NA>,<NA>,[]
4196,Homestuck,Jade: Attempt to deploy catchalogue disk drive.,"Again, the name of the thing was right there i...",003416,2010-02-28 23:36:45,/storyfiles/hs2/01516.gif,[003417],003415,1516,https://homestuck.com/story/1516,"[Side 1, Part 1, Act 4, Dave Strider, Scratche...","[Dave, Jade]","[[Dave: EGG!], [Jade: EGG!]]",<NA>,<NA>,<NA>,⬡ 798/2000 → 788,<NA>,[]
3417,Homestuck,==>,You are accosted by a friendly MUTANT KITTEN.,002807,2009-11-23 21:20:05,/storyfiles/hs2/00907.gif,[002808],002806,907,https://homestuck.com/story/907,"[Part 1, Act 3, Rose Lalonde, Squiddle Shirt, ...",[Rose],"[[Rose: Find child's bedroom, Vodka Mutini]]",<NA>,<NA>,<NA>,<NA>,<NA>,[]
2442,Homestuck,John: Leave a surprise for the mailman.,N...<br /><br />No!,001984,2009-04-24 23:16:08,/storyfiles/hs2/00084.gif,[001985],001983,84,https://homestuck.com/story/84,"[Part 1, Act 1, John Egbert, Green Slime Ghost...",[John],"[[John: Be introduced, John: Mess around]]",<NA>,<NA>,<NA>,<NA>,<NA>,[]
3572,Homestuck,WV: Eat letter and envelope.,Will you cut that out! You have company.,002936,2009-12-20 03:48:10,/storyfiles/hs2/01036.gif,[002937],002935,1036,https://homestuck.com/story/1036,"[Part 1, Act 3, Lined Mode, Carapacian, Dersit...",[Warweary Villein],[[Warweary Villein: WV: Descend.]],<NA>,<NA>,<NA>,NIBBLE NIBBLE,<NA>,[]


In [444]:
# For the dataset we'll need to make a new dataframe with only the relevant data
# We don't need viz_story, timestamp, next, previous, viz_page_number, viz_link
# We'll keep the rest, and remove the tag later when it's time

tagging_dataset = four_acts_hs_df.drop(columns=['viz_story', 'timestamp', 'next', 'previous', 'viz_page_number', 'viz_link'])

tagging_dataset.sample(5)

,title,content,page_id,src,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,theme,flag
2879,Dave: Open Complete Bullshit.,This is COMPLETE BULLSHIT.,002358,/storyfiles/hs2/00458_2.gif,"[Act 2, Part 1, Screen]",[Dave],"[[Dave: Go on Bro's computer, Dave: Talk to Jo...",<NA>,<NA>,<NA>,<NA>,<NA>,[]
2650,JOHN. RECYCLE THE GRIST AS WAS DICTATED BY YOU...,John cannot do anything with the GRIST as of t...,002160,/storyfiles/hs2/00260.gif,"[Act 2, Part 1, John Egbert, Green Slime Ghost...","[John, Rose, Warweary Villein]","[[John: <==, John: BOY., John: YOU THERE. BOY....",<NA>,<NA>,<NA>,JOHN: ? 0/20,<NA>,[]
4385,AH: I didn't read any of that. Do something le...,&gt; MSPA Reader: Shut the hell up.,003575,/storyfiles/hs2/01675.gif,"[Part 1, Act 4, Andrew Hussie, Hussie Manor, H...",[],[],<NA>,<NA>,<NA>,<NA>,<NA>,[]
3766,[I] ==>,"He says he's got <span style=""color: #2ed73a"">...",003076,/storyfiles/hs2/01176.gif,"[Part 1, Intermission, Midnight Crew, Jack Noi...","[Clubs Deuce, Spades Slick]","[[Clubs Deuce: Inform SS of capture of Doze, C...",<NA>,<NA>,<NA>,<NA>,<NA>,[]
3751,[I] SS: Open war chest.,<NA>,003062,/storyfiles/hs2/01162.gif,"[Part 1, Intermission, Felt Manor, Stairs, Mid...",[Spades Slick],"[[Spades Slick: Enter mansion, Spades Slick: C...",<NA>,<NA>,<NA>,<NA>,<NA>,[]


In [465]:
"""
Now let's turn it into OpenAI's JSONL format, an example of which is here:

```jsonl
{
  "messages": [
    { "role": "system", "content": "You are an assistant that identifies uncommon cheeses." },
    { "role": "user", "content": "What is this cheese?" },
    { "role": "user", "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/3/36/Danbo_Cheese.jpg"
          }
        }
      ]
    },
    { "role": "assistant", "content": "Danbo" }
  ]
}
```

Each row of the dataset will turn into something here. 
The system prompt will contain an example of the input data, so let's deal with that later.
The first user content will be the metadata minus the image and the tags.
The second user content will be the image URL, which we can once again get from https://homestuck.com/images plus the src column. No need to base64 encode it.
The assistant content will be the tags column.
"""

system_prompt = """You are a professional image tagger.
Your current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.
You will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more metadata.
The image alongside will be the panel itself. Here's an example of the metadata input:
```
{"title": "Jade: Keep going.", "content": "Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.", "page_id": "002753", "src": "/storyfiles/hs2/00853_1.gif", "pov_characters": ["Jade"], "pov_commands": [["Jade: Proceed through house", "Jade: Be trolled by Karkat"]], "transcript_class": null, "transcript_style": null, "transcript_tag": null, "transcript": null, "theme": null, "flag": []}
```
And an example of your tagged output, which must be a valid JSON list:
```
["Jade Harley", "Jade's House", "Blue Sun Shirt", "Stairs", "Dead", "Light Symbol"]
```
"""

def create_openai_jsonl_entry(row: pd.Series) -> dict:
    """
    Creates an OpenAI JSONL entry from a row of the tagging dataset.

    Args:
        row (pd.Series): A row of the tagging dataset.

    Returns:
        dict: An OpenAI JSONL entry.
    """
    # We must remove the organizational tags like "Act 1" and "Part 1" from the tags, because
    # the model cannot predict them
    usable_tags = [tag for tag in row['tags'] if "Act " not in tag and "Part " not in tag]
    # Tags will also be stringified so the model can predict them
    usable_tags = json.dumps(usable_tags)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": json.dumps(row.drop('tags').to_dict())},
         {"role": "user", "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"https://homestuck.com/images{row['src']}"
                }
            }
        ]},
        {"role": "assistant", "content": usable_tags}
    ]
    return {"messages": messages}

# Test it with one row
create_openai_jsonl_entry(tagging_dataset.iloc[1000])

{'messages': [{'role': 'system',
   'content': 'You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more metadata.\nThe image alongside will be the panel itself. Here\'s an example of the metadata input:\n```\n{"title": "Jade: Keep going.", "content": "Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.", "page_id": "002753", "src": "/storyfiles/hs2/00853_1.gif", "pov_characters": ["Jade"], "pov_commands": [["Jade: Proceed through house", "Jade: Be trolled by Karkat"]], "transcript_class": null, "transcript_style": null, "transcript_tag": null, "transcript": null, "theme": null, "flag": []}\n```\nAnd an example of your tagged output, which must be a valid JSON l

In [474]:
# OpenAI's platform lets you upload two jsonl files, one with the training data and another with the validation data
# We'll split the dataset into 80% training and 20% validation, but we need to shuffle it first

shuffled_tagging_dataset = tagging_dataset.sample(frac=1, random_state=42)
training_size = int(len(shuffled_tagging_dataset) * 0.8)
training_data = shuffled_tagging_dataset.iloc[:training_size]
validation_data = shuffled_tagging_dataset.iloc[training_size:]

# Remove the training and validation files if they don't already exist
# Remember we're storing them in output, as "hs_tagging_tranining_data.jsonl" and "hs_tagging_validation_data.jsonl"

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
    
# Remove the files if they exist
training_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_training_data.jsonl")
validation_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_validation_data.jsonl")

if os.path.exists(training_file):
    os.remove(training_file)
if os.path.exists(validation_file):
    os.remove(validation_file)

# Create the training and validation jsonl files
with open(training_file, 'w') as training_jsonl, open(validation_file, 'w') as validation_jsonl:
    for _, row in training_data.iterrows():
        training_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Training data written to {training_file}")
    for _, row in validation_data.iterrows():
        validation_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Validation data written to {validation_file}")
        
# Let's check the first few lines of the training file
with open(training_file, 'r') as file:
    for _ in range(5):
        print(file.readline())
        
# Let's check the first few lines of the validation file
with open(validation_file, 'r') as file:
    for _ in range(5):
        print(file.readline())

Training data written to ../output\hs_tagging_training_data.jsonl
Validation data written to ../output\hs_tagging_validation_data.jsonl
{"messages": [{"role": "system", "content": "You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more metadata.\nThe image alongside will be the panel itself. Here's an example of the metadata input:\n```\n{\"title\": \"Jade: Keep going.\", \"content\": \"Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.\", \"page_id\": \"002753\", \"src\": \"/storyfiles/hs2/00853_1.gif\", \"pov_characters\": [\"Jade\"], \"pov_commands\": [[\"Jade: Proceed through house\", \"Jade: Be trolled by Karkat\"]], \"transcript_class\": null, \"transcr

In [427]:
# Queries to send to the annotation model
batch_queries = []
system_prompt =  """You are a professional image annotator.
Your current project is annotating the panels of the webcomic Homestuck. You need to write a textual description as well as a list of location and character tags.
Your input will some metadata like this:
{
    "src": "005624.gif",
    "title": "Jane: Reply",
    "pov_characters": ["Jane Crocker", "Caliborn"],
    "transcript": ["...", "HELP"]
}
This is an example output:
{
    "": ["Jane Crocker"],
    "locations": ["Land of Crypts and Helium", "Jane's House"],  # If you don't know the location, just leave it out
    "description": "Jane Crocker stands in the middle of her room, next to her bed. Jane is wearing a gray shirt with a blue monster logo on it, as well as a blue skirt. The room contains posters of movies. Outside the window we can see the Land of Crypts and Helium, a gray planet with multicolored flowers. There's a text bubble with '...' pointing to her head.",
}
You should write verbose descriptions that will be useful for people who can't see the image, as well as for training image models.
No talk; just go.
"""

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    

def annotate_panel(panel_data: dict, image_path: str) -> dict:
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": [
            {
                "type": "text",
                "text": json.dumps(panel_data)
            },
            {
                "type": "image_url", 
                "image_url": {
                    "url": f"data:image/png;base64,{encode_image(image_path)}"
                }
            }
        ]}
    ]

    response = openai_client.chat.completions.create(
        model=MODEL_ID,
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message.content

def get_panel_data(panel_id: str) -> dict:
    # We get metadata from a variety of sources
    # src: the image file name
    # title, page_content: from the json archive
    # pov_characters: from the POV extension
    # transcript: from the ReadMSPA transcripts
    pass
    

# We load a test panel, from HS_PANELS_FOLDER, 01691.gif which should depict a sleeping Rose and an awake John
test_panel = os.path.join(HS_PANELS_FOLDER, "01691.gif")
test_panel_data = {
    "src": "01691.gif",
    "title": "John: Get up.",
    "page_content": """Despite the pandemonium of your entrance, Rose is still sound asleep. She must be really tuckered out!
<br>
<br>It looks like this little guy is awake and ready for action though. He is adorable. You decide to name him Dr. Meowgon Spengler.""",
    "pov_characters": ["John Egbert", "Rose Lalonde"],
    "transcript": ["Z"],
}

# We'll use the OpenAI API to annotate the panel
# This is a test, so we'll just print the output
# print(annotate_panel(test_panel_data, test_panel))

# Textual datasets

We output datasets for Homestuck, MS Paint Adventures as a whole, and all of Andrew Hussie's works.

The format is .jsonl